---
title: Assignment 04
author:
  - name: Ava Godsy
    affiliations:
      - id: bu
        name: Boston University
        city: Boston
        state: MA
number-sections: true
date: today
date-modified: today
date-format: long
format:
  html:
    theme: cerulean
    toc: true
    toc-depth: 2
  docx: default
  pdf: default
execute:
  echo: false
  eval: false
  freeze: auto
---

# AI Disclosure
For each of the following, see the Appendix: AI Prompts section for the prompts and responses used to generate the code. I used Claude to complete this assignment. I found that the code blocks tended to be longer and more complex than what Chat GPT created. Claude also provided lengthy context for why the code was structured the way it was. I found this helpful for understanding the code, but it also made the responses much longer. I had to be more specific in my prompts to get the exact code I wanted.

# Load the Dataset

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
import plotly.express as px
import plotly.io as pio
import numpy as np
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.sql.functions import col, pow as spark_pow, when, trim
from pyspark.ml.regression import LinearRegression
from pyspark.sql.functions import when, trim
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors, DenseVector
from scipy import stats as scipy_stats
from pyspark.ml.regression import RandomForestRegressor
import plotly.graph_objects as go
import os

np.random.seed(42)

pio.renderers.default = "notebook+notebook_connected+vscode"

# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("lightcast_job_postings.csv")

# Show Schema and Sample Data
# print("---This is Diagnostic check, No need to print it in the final doc---")

# df.printSchema() # comment this line when rendering the submission
# df.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 23:11:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Feature Engineering

In [2]:
# Assuming df is your existing DataFrame
# Step 1: Combine remote type values before cleaning

df_processed = df.withColumn('REMOTE_TYPE_NAME',
    when((col('REMOTE_TYPE_NAME').isNull()) | 
         (trim(col('REMOTE_TYPE_NAME')) == '[None]') |
         (trim(col('REMOTE_TYPE_NAME')) == 'Not Remote' ) |
         (trim(col('REMOTE_TYPE_NAME')) == 'Onsite'), 'Onsite')
    .when((col('REMOTE_TYPE_NAME') == 'Hybrid Remote'), 'Hybrid')
    .otherwise(col('REMOTE_TYPE_NAME'))
)

print("=== REMOTE_TYPE_NAME VALUE COUNTS AFTER COMBINING ===")
df_processed.groupBy('REMOTE_TYPE_NAME').count().orderBy('count', ascending=False).show()

# Step 2: Drop rows with missing values in target and key features
selected_columns = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                   'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY']

df_clean = df_processed.select(selected_columns).dropna()

print("Original DataFrame count:", df.count())
print("Cleaned DataFrame count:", df_clean.count())
print("\nCleaned DataFrame Schema:")
df_clean.printSchema()

# Step 2: Create squared feature for MIN_YEARS_EXPERIENCE
df_clean = df_clean.withColumn('MIN_YEARS_EXPERIENCE_SQ', 
                               spark_pow(col('MIN_YEARS_EXPERIENCE'), 2))

print("\nDataFrame with squared feature:")
df_clean.show(5)

# Step 3: Create Pipeline for encoding and feature assembly

# StringIndexer for categorical variables
msa_indexer = StringIndexer(inputCol='MSA_NAME', 
                            outputCol='MSA_NAME_INDEX',
                            handleInvalid='keep')

remote_indexer = StringIndexer(inputCol='REMOTE_TYPE_NAME', 
                               outputCol='REMOTE_TYPE_NAME_INDEX',
                               handleInvalid='keep')

# OneHotEncoder for categorical variables
msa_encoder = OneHotEncoder(inputCol='MSA_NAME_INDEX', 
                           outputCol='MSA_NAME_VEC',
                           dropLast=True)

remote_encoder = OneHotEncoder(inputCol='REMOTE_TYPE_NAME_INDEX', 
                              outputCol='REMOTE_TYPE_NAME_VEC',
                              dropLast=True)

# VectorAssembler for basic features
feature_cols = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
               'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler = VectorAssembler(inputCols=feature_cols, 
                            outputCol='features',
                            handleInvalid='keep')

# VectorAssembler for polynomial features
poly_feature_cols = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                    'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                    'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

poly_assembler = VectorAssembler(inputCols=poly_feature_cols, 
                                outputCol='features_poly',
                                handleInvalid='keep')

# Create Pipeline
pipeline = Pipeline(stages=[
    msa_indexer,
    remote_indexer,
    msa_encoder,
    remote_encoder,
    assembler,
    poly_assembler
])

# Fit and transform the data
pipeline_model = pipeline.fit(df_clean)
df_transformed = pipeline_model.transform(df_clean)

print("\nTransformed DataFrame with all features:")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY',
                     'features', 'features_poly').show(5, truncate=False)


=== REMOTE_TYPE_NAME VALUE COUNTS AFTER COMBINING ===


+----------------+-----+
|REMOTE_TYPE_NAME|count|
+----------------+-----+
|          Onsite|57741|
|          Remote|12497|
|          Hybrid| 2260|
+----------------+-----+



Original DataFrame count: 72498


Cleaned DataFrame count: 3596

Cleaned DataFrame Schema:
root
 |-- MIN_YEARS_EXPERIENCE: integer (nullable = true)
 |-- MAX_YEARS_EXPERIENCE: integer (nullable = true)
 |-- SALARY_FROM: integer (nullable = true)
 |-- MSA_NAME: string (nullable = true)
 |-- REMOTE_TYPE_NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)


DataFrame with squared feature:
+--------------------+--------------------+-----------+--------------------+----------------+------+-----------------------+
|MIN_YEARS_EXPERIENCE|MAX_YEARS_EXPERIENCE|SALARY_FROM|            MSA_NAME|REMOTE_TYPE_NAME|SALARY|MIN_YEARS_EXPERIENCE_SQ|
+--------------------+--------------------+-----------+--------------------+----------------+------+-----------------------+
|                   2|                   2|      79500|New York-Newark-J...|          Onsite| 92962|                    4.0|
|                   2|                   2|      75026|         Jackson, MS|          Onsite| 75026|                    4.0|
| 


Transformed DataFrame with all features:
+--------------------+-----------------------+--------------------+-----------+-------------------------------------+----------------+------+-----------------------------------------------+-----------------------------------------------------+
|MIN_YEARS_EXPERIENCE|MIN_YEARS_EXPERIENCE_SQ|MAX_YEARS_EXPERIENCE|SALARY_FROM|MSA_NAME                             |REMOTE_TYPE_NAME|SALARY|features                                       |features_poly                                        |
+--------------------+-----------------------+--------------------+-----------+-------------------------------------+----------------+------+-----------------------------------------------+-----------------------------------------------------+
|2                   |4.0                    |2                   |79500      |New York-Newark-Jersey City, NY-NJ-PA|Onsite          |92962 |(217,[0,1,2,3,214],[2.0,2.0,79500.0,1.0,1.0])  |(218,[0,1,2,3,4,215],[2.0,4.0,2.0,795

# Train/Test Split

In [3]:
# Step 4: Split data into training and testing sets
# Using 70-30 split for better evaluation capability
# Justification:
# - 70% training: Provides sufficient data for model to learn patterns
# - 30% testing: Larger test set gives more reliable performance metrics
# - Good balance for datasets with moderate size (thousands of records)
# - Seed=42 ensures reproducibility across runs
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)

print(f"\n=== DATA SPLIT SUMMARY ===")
print(f"Training set count: {train_data.count()} ({train_data.count()/df_transformed.count()*100:.1f}%)")
print(f"Testing set count: {test_data.count()} ({test_data.count()/df_transformed.count()*100:.1f}%)")
print(f"Random seed: 42 (for reproducibility)")
print("\nSplit Justification:")
print("• 70-30 split provides robust model evaluation")
print("• Larger test set (30%) improves confidence in performance metrics")
print("• Balanced approach for moderate-sized datasets")
print("• Alternative splits: 80-20 for large datasets, 60-40 for small datasets")

# Step 5: Show final structure
print("\n=== FINAL DATAFRAME STRUCTURE ===")
df_transformed.printSchema()

print("\n=== SAMPLE OF FINAL DATA ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                     'SALARY', 'features_poly').show(10)

# Display feature statistics
print("\n=== FEATURE STATISTICS ===")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'SALARY').describe().show()

# Optional: Show unique values in categorical columns
print("\n=== CATEGORICAL VARIABLE COUNTS ===")
print("MSA_NAME unique values:", df_clean.select('MSA_NAME').distinct().count())
print("REMOTE_TYPE_NAME unique values:", df_clean.select('REMOTE_TYPE_NAME').distinct().count())

# Save transformed data for future use (optional)
# df_transformed.write.parquet("transformed_salary_data.parquet", mode='overwrite')

print("\n✓ Data preprocessing pipeline completed successfully!")
print("✓ Ready for model training with 'features_poly' as input and 'SALARY' as target")


=== DATA SPLIT SUMMARY ===


Training set count: 2574 (71.6%)


Testing set count: 1022 (28.4%)
Random seed: 42 (for reproducibility)

Split Justification:
• 70-30 split provides robust model evaluation
• Larger test set (30%) improves confidence in performance metrics
• Balanced approach for moderate-sized datasets
• Alternative splits: 80-20 for large datasets, 60-40 for small datasets

=== FINAL DATAFRAME STRUCTURE ===
root
 |-- MIN_YEARS_EXPERIENCE: integer (nullable = true)
 |-- MAX_YEARS_EXPERIENCE: integer (nullable = true)
 |-- SALARY_FROM: integer (nullable = true)
 |-- MSA_NAME: string (nullable = true)
 |-- REMOTE_TYPE_NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- MIN_YEARS_EXPERIENCE_SQ: double (nullable = true)
 |-- MSA_NAME_INDEX: double (nullable = false)
 |-- REMOTE_TYPE_NAME_INDEX: double (nullable = false)
 |-- MSA_NAME_VEC: vector (nullable = true)
 |-- REMOTE_TYPE_NAME_VEC: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- features_poly: vector (nullable = true)


=== SAMPLE OF FI

25/10/08 23:12:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------------------+-----------------------+--------------------+------------------+------------------+
|summary|MIN_YEARS_EXPERIENCE|MIN_YEARS_EXPERIENCE_SQ|MAX_YEARS_EXPERIENCE|       SALARY_FROM|            SALARY|
+-------+--------------------+-----------------------+--------------------+------------------+------------------+
|  count|                3596|                   3596|                3596|              3596|              3596|
|   mean|  3.6384872080088986|       18.9972191323693|  3.6384872080088986| 91714.32619577309| 107798.5881535039|
| stddev|   2.400048294044211|     24.190746240439758|   2.400048294044211|32683.349277662266|36636.119374840724|
|    min|                   0|                    0.0|                   0|             14000|             31640|
|    max|                  12|                  144.0|                  12|            324000|            338750|
+-------+--------------------+-----------------------+--------------------+-------------

MSA_NAME unique values: 211


REMOTE_TYPE_NAME unique values: 3

✓ Data preprocessing pipeline completed successfully!
✓ Ready for model training with 'features_poly' as input and 'SALARY' as target


# Linear Regression

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
from pyspark.sql.functions import col, pow as spark_pow, when, trim
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.linalg import Vectors, DenseVector
import pandas as pd
import numpy as np
from scipy import stats as scipy_stats

# Initialize Spark Session (if not already created)
spark = SparkSession.builder.appName("SalaryPrediction").getOrCreate()

# Assuming df is your existing DataFrame
# Step 1: Combine remote type values before cleaning
df_processed = df.withColumn('REMOTE_TYPE_NAME',
    when((col('REMOTE_TYPE_NAME').isNull()) | 
         (trim(col('REMOTE_TYPE_NAME')) == '[None]') |
         (trim(col('REMOTE_TYPE_NAME')) == 'Not Remote' ) |
         (trim(col('REMOTE_TYPE_NAME')) == 'Onsite'), 'Onsite')
    .when((col('REMOTE_TYPE_NAME') == 'Hybrid Remote'), 'Hybrid')
    .otherwise(col('REMOTE_TYPE_NAME'))
)

print("=== REMOTE_TYPE_NAME VALUE COUNTS AFTER COMBINING ===")
df_processed.groupBy('REMOTE_TYPE_NAME').count().orderBy('count', ascending=False).show()

# Step 2: Drop rows with missing values in target and key features
selected_columns = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                   'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY']

df_clean = df_processed.select(selected_columns).dropna()

print("Original DataFrame count:", df.count())
print("Cleaned DataFrame count:", df_clean.count())
print("\nCleaned DataFrame Schema:")
df_clean.printSchema()

# Step 2: Create squared feature for MIN_YEARS_EXPERIENCE
df_clean = df_clean.withColumn('MIN_YEARS_EXPERIENCE_SQ', 
                               spark_pow(col('MIN_YEARS_EXPERIENCE'), 2))

print("\nDataFrame with squared feature:")
df_clean.show(5)

# Step 3: Create Pipeline for encoding and feature assembly

# StringIndexer for categorical variables
msa_indexer = StringIndexer(inputCol='MSA_NAME', 
                            outputCol='MSA_NAME_INDEX',
                            handleInvalid='keep')

remote_indexer = StringIndexer(inputCol='REMOTE_TYPE_NAME', 
                               outputCol='REMOTE_TYPE_NAME_INDEX',
                               handleInvalid='keep')

# OneHotEncoder for categorical variables
msa_encoder = OneHotEncoder(inputCol='MSA_NAME_INDEX', 
                           outputCol='MSA_NAME_VEC',
                           dropLast=True)

remote_encoder = OneHotEncoder(inputCol='REMOTE_TYPE_NAME_INDEX', 
                              outputCol='REMOTE_TYPE_NAME_VEC',
                              dropLast=True)

# VectorAssembler for basic features
feature_cols = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
               'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler = VectorAssembler(inputCols=feature_cols, 
                            outputCol='features',
                            handleInvalid='keep')

# VectorAssembler for polynomial features
poly_feature_cols = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                    'MAX_YEARS_EXPERIENCE', 'SALARY_FROM',
                    'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

poly_assembler = VectorAssembler(inputCols=poly_feature_cols, 
                                outputCol='features_poly',
                                handleInvalid='keep')

# Create Pipeline
pipeline = Pipeline(stages=[
    msa_indexer,
    remote_indexer,
    msa_encoder,
    remote_encoder,
    assembler,
    poly_assembler
])

# Fit and transform the data
pipeline_model = pipeline.fit(df_clean)
df_transformed = pipeline_model.transform(df_clean)

print("\nTransformed DataFrame with all features:")
df_transformed.select('MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 
                     'MAX_YEARS_EXPERIENCE', 'SALARY_FROM', 
                     'MSA_NAME', 'REMOTE_TYPE_NAME', 'SALARY',
                     'features', 'features_poly').show(5, truncate=False)

# Step 4: Split data into training and testing sets
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)

print(f"\n=== DATA SPLIT SUMMARY ===")
print(f"Training set count: {train_data.count()} ({train_data.count()/df_transformed.count()*100:.1f}%)")
print(f"Testing set count: {test_data.count()} ({test_data.count()/df_transformed.count()*100:.1f}%)")

# ============================================================================
# STEP 5: TRAIN LINEAR REGRESSION MODEL
# ============================================================================
print("\n" + "="*80)
print("TRAINING LINEAR REGRESSION MODEL")
print("="*80)

# CRITICAL ISSUE RESOLUTION: 
# The 'features' column includes SALARY_FROM which is highly correlated with SALARY
# This creates MULTICOLLINEARITY and DATA LEAKAGE issues:
# 1. SALARY_FROM is derived from the same job posting as SALARY (target variable)
# 2. Including it violates the independence assumption
# 3. It artificially inflates R² and makes the model unusable for real predictions
# 
# SOLUTION: Create a new feature vector WITHOUT SALARY_FROM

print("\n⚠️  IDENTIFYING THE KEY ISSUE:")
print("The 'features' column includes SALARY_FROM, which creates DATA LEAKAGE!")
print("SALARY_FROM is part of the same salary range as our target (SALARY).")
print("This violates ML principles and makes the model unrealistic.\n")

# Create a new assembler WITHOUT SALARY_FROM
feature_cols_clean = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE',
                      'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler_clean = VectorAssembler(inputCols=feature_cols_clean, 
                                  outputCol='features_clean',
                                  handleInvalid='keep')

# Transform data with clean features
df_train = assembler_clean.transform(train_data)
df_test = assembler_clean.transform(test_data)

print("✓ Created 'features_clean' column WITHOUT SALARY_FROM")
print(f"  Features included: {feature_cols_clean}\n")

# Initialize Linear Regression model
lr = LinearRegression(
    featuresCol='features_clean',
    labelCol='SALARY',
    maxIter=100,
    regParam=0.0,
    elasticNetParam=0.0,
    standardization=True
)

# Train the model
print("Training Linear Regression model...")
lr_model = lr.fit(df_train)
print("✓ Model training completed!\n")

# ============================================================================
# STEP 6: MAKE PREDICTIONS AND EVALUATE
# ============================================================================
print("="*80)
print("MODEL EVALUATION")
print("="*80 + "\n")

# Make predictions on test data
predictions = lr_model.transform(df_test)

# Display sample predictions
print("=== SAMPLE PREDICTIONS ===")
predictions.select('SALARY', 'prediction', 'MIN_YEARS_EXPERIENCE', 
                   'MAX_YEARS_EXPERIENCE', 'MSA_NAME', 'REMOTE_TYPE_NAME').show(10)

# ============================================================================
# STEP 7: EXTRACT MODEL COEFFICIENTS AND CALCULATE STATISTICS MANUALLY
# ============================================================================
print("\n" + "="*80)
print("MODEL COEFFICIENTS AND STATISTICS")
print("="*80 + "\n")

# Get model summary
summary = lr_model.summary

# Extract basic metrics
intercept = lr_model.intercept
coefficients = lr_model.coefficients
r2 = summary.r2
rmse = summary.rootMeanSquaredError
mae = summary.meanAbsoluteError

print(f"Intercept: ${intercept:,.2f}")
print(f"R² (R-squared): {r2:.4f}")
print(f"RMSE (Root Mean Squared Error): ${rmse:,.2f}")
print(f"MAE (Mean Absolute Error): ${mae:,.2f}")

# ============================================================================
# MANUAL CALCULATION OF COEFFICIENT STATISTICS
# ============================================================================
print("\n" + "="*80)
print("CALCULATING COEFFICIENT STATISTICS MANUALLY")
print("="*80 + "\n")

print("Extracting feature matrix and target values from training data...")

# Collect training data for manual statistics calculation
# WARNING: Only do this if dataset is not too large (< 100K rows recommended)
train_count = df_train.count()
print(f"Training set size: {train_count:,} rows")

if train_count > 100000:
    print("⚠️  Warning: Large dataset. Manual statistics calculation may be slow.")
    print("   Consider using a sample for coefficient statistics.\n")

# Extract features and labels
train_features = np.array(df_train.select('features_clean').rdd.map(lambda row: row[0].toArray()).collect())
train_labels = np.array(df_train.select('SALARY').rdd.map(lambda row: row[0]).collect())

print(f"Feature matrix shape: {train_features.shape}")
print(f"Label vector shape: {train_labels.shape}")

# Get predictions on training data for residuals
train_predictions = lr_model.transform(df_train)
train_pred_values = np.array(train_predictions.select('prediction').rdd.map(lambda row: row[0]).collect())

# Calculate residuals
residuals = train_labels - train_pred_values
n = len(train_labels)
k = train_features.shape[1]  # number of features
df_residual = n - k - 1  # degrees of freedom

# Calculate residual standard error
rse = np.sqrt(np.sum(residuals**2) / df_residual)

print(f"\nResidual Standard Error: ${rse:,.2f}")
print(f"Degrees of Freedom: {df_residual}")

# Calculate variance-covariance matrix
# Var(β) = σ² * (X'X)^(-1)
try:
    X = train_features
    XtX = np.dot(X.T, X)
    XtX_inv = np.linalg.inv(XtX)
    
    # Variance-covariance matrix
    var_covar_matrix = (rse**2) * XtX_inv
    
    # Standard errors are square roots of diagonal elements
    std_errors = np.sqrt(np.diag(var_covar_matrix))
    
    # Calculate t-values
    coef_array = np.array(coefficients.toArray())
    t_values = coef_array / std_errors
    
    # Calculate p-values (two-tailed test)
    p_values = 2 * (1 - scipy_stats.t.cdf(np.abs(t_values), df_residual))
    
    # Calculate 95% confidence intervals
    t_critical = scipy_stats.t.ppf(0.975, df_residual)  # 97.5th percentile for two-tailed
    ci_lower = coef_array - t_critical * std_errors
    ci_upper = coef_array + t_critical * std_errors
    
    stats_available = True
    print("✓ Coefficient statistics calculated successfully!\n")
    
except np.linalg.LinAlgError as e:
    print(f"❌ Error calculating statistics: {e}")
    print("   This may happen with singular matrices or perfect multicollinearity.\n")
    stats_available = False
    std_errors = [None] * len(coefficients)
    t_values = [None] * len(coefficients)
    p_values = [None] * len(coefficients)
    ci_lower = [None] * len(coefficients)
    ci_upper = [None] * len(coefficients)

# ============================================================================
# CREATE COEFFICIENT TABLE
# ============================================================================

# Create feature names for interpretation
num_msa_categories = df_clean.select('MSA_NAME').distinct().count() - 1
num_remote_categories = df_clean.select('REMOTE_TYPE_NAME').distinct().count() - 1

feature_names = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE']
feature_names += [f'MSA_{i}' for i in range(num_msa_categories)]
feature_names += [f'REMOTE_{i}' for i in range(num_remote_categories)]

# Create DataFrame for coefficient analysis
coef_data = []
for i, (name, coef) in enumerate(zip(feature_names, coefficients)):
    row_data = {
        'Feature': name,
        'Coefficient': float(coef)
    }
    
    if stats_available:
        row_data.update({
            'Std_Error': float(std_errors[i]),
            'T_Value': float(t_values[i]),
            'P_Value': float(p_values[i]),
            'CI_Lower': float(ci_lower[i]),
            'CI_Upper': float(ci_upper[i]),
            'Significant': '***' if p_values[i] < 0.001 else '**' if p_values[i] < 0.01 else '*' if p_values[i] < 0.05 else 'No'
        })
    
    coef_data.append(row_data)

# Convert to Pandas for better display
coef_df = pd.DataFrame(coef_data)

print("\n=== COEFFICIENT ANALYSIS TABLE ===")
print(coef_df.to_string(index=False))

# ============================================================================
# STEP 8: INTERPRET RESULTS
# ============================================================================
print("\n" + "="*80)
print("MODEL INTERPRETATION")
print("="*80 + "\n")

print("📊 COEFFICIENTS INTERPRETATION:")
print("-" * 80)

if stats_available:
    for i, row in coef_df.iterrows():
        if i < 2:  # Only interpret the main numerical features
            name = row['Feature']
            coef = row['Coefficient']
            p_val = row['P_Value']
            sig = row['Significant']
            
            print(f"\n{name}:")
            print(f"  • Coefficient: ${coef:,.2f} {sig}")
            print(f"  • Interpretation: For each additional year of experience,")
            print(f"    salary {'increases' if coef > 0 else 'decreases'} by ${abs(coef):,.2f} (all else equal)")
            print(f"  • Statistical Significance: {'Significant' if sig != 'No' else 'Not significant'} (p={p_val:.4f})")
            print(f"  • 95% CI: [${row['CI_Lower']:,.2f}, ${row['CI_Upper']:,.2f}]")
            if p_val < 0.05:
                print(f"  • Conclusion: This effect is statistically significant at the 5% level")
            else:
                print(f"  • Conclusion: This effect is NOT statistically significant")
else:
    for i, row in coef_df.iterrows():
        if i < 2:
            name = row['Feature']
            coef = row['Coefficient']
            
            print(f"\n{name}:")
            print(f"  • Coefficient: ${coef:,.2f}")
            print(f"  • Interpretation: For each additional year of experience,")
            print(f"    salary {'increases' if coef > 0 else 'decreases'} by ${abs(coef):,.2f} (all else equal)")

print("\n" + "-" * 80)
print("\n📈 MODEL PERFORMANCE METRICS:")
print("-" * 80)

print(f"\n1. R² (R-squared) = {r2:.4f}")
print(f"   • Interpretation: The model explains {r2*100:.2f}% of the variance in salary")
if r2 > 0.7:
    print(f"   • Assessment: Strong explanatory power")
elif r2 > 0.5:
    print(f"   • Assessment: Moderate explanatory power")
elif r2 > 0.3:
    print(f"   • Assessment: Weak but meaningful explanatory power")
else:
    print(f"   • Assessment: Poor explanatory power - consider adding more features")

print(f"\n2. RMSE (Root Mean Squared Error) = ${rmse:,.2f}")
print(f"   • Interpretation: On average, predictions deviate by ${rmse:,.2f}")
print(f"   • Assessment: Predictions are typically off by ~${rmse:,.0f}")

print(f"\n3. MAE (Mean Absolute Error) = ${mae:,.2f}")
print(f"   • Interpretation: The average absolute prediction error is ${mae:,.2f}")
ratio = mae / rmse if rmse > 0 else 0
print(f"   • MAE/RMSE Ratio: {ratio:.3f}")
if ratio < 0.8:
    print(f"   • Large errors present (outliers) since MAE << RMSE")
elif ratio < 0.9:
    print(f"   • Some large errors present")
else:
    print(f"   • Errors are relatively uniform")

# Calculate additional evaluators
evaluator_r2 = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="mae")

test_r2 = evaluator_r2.evaluate(predictions)
test_rmse = evaluator_rmse.evaluate(predictions)
test_mae = evaluator_mae.evaluate(predictions)

print("\n" + "-" * 80)
print("\n🎯 TEST SET PERFORMANCE:")
print("-" * 80)
print(f"Test R²: {test_r2:.4f}")
print(f"Test RMSE: ${test_rmse:,.2f}")
print(f"Test MAE: ${test_mae:,.2f}")

# Compare training vs test performance
print("\n📊 TRAINING vs TEST COMPARISON:")
print("-" * 80)
print(f"Training R²: {r2:.4f} | Test R²: {test_r2:.4f} | Difference: {abs(r2-test_r2):.4f}")
if abs(r2 - test_r2) < 0.05:
    print("✓ Excellent generalization - minimal overfitting")
elif abs(r2 - test_r2) < 0.10:
    print("✓ Good generalization - acceptable overfitting")
elif abs(r2 - test_r2) < 0.15:
    print("⚠ Moderate overfitting detected - consider regularization")
else:
    print("❌ Significant overfitting - model may not generalize well")
    print("   Consider: reducing features, adding regularization, or collecting more data")

if stats_available:
    print("\n" + "-" * 80)
    print("\n🔍 STATISTICAL INSIGHTS:")
    print("-" * 80)
    sig_features = coef_df[coef_df['Significant'] != 'No'] if 'Significant' in coef_df.columns else pd.DataFrame()
    if len(sig_features) > 0:
        print(f"Number of significant features (p < 0.05): {len(sig_features)}")
        print(f"Total features: {len(coef_df)}")
        print(f"Percentage significant: {len(sig_features)/len(coef_df)*100:.1f}%")
    
    print(f"\nAdjusted R²: {1 - (1-r2)*(n-1)/(n-k-1):.4f}")
    print(f"  • Accounts for number of predictors")
    print(f"  • Better metric for comparing models with different numbers of features")

print("\n" + "="*80)
print("✓ MODEL TRAINING AND EVALUATION COMPLETED!")
print("="*80)

25/10/08 23:13:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


=== REMOTE_TYPE_NAME VALUE COUNTS AFTER COMBINING ===


+----------------+-----+
|REMOTE_TYPE_NAME|count|
+----------------+-----+
|          Onsite|57741|
|          Remote|12497|
|          Hybrid| 2260|
+----------------+-----+



Original DataFrame count: 72498


Cleaned DataFrame count: 3596

Cleaned DataFrame Schema:
root
 |-- MIN_YEARS_EXPERIENCE: integer (nullable = true)
 |-- MAX_YEARS_EXPERIENCE: integer (nullable = true)
 |-- SALARY_FROM: integer (nullable = true)
 |-- MSA_NAME: string (nullable = true)
 |-- REMOTE_TYPE_NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)


DataFrame with squared feature:
+--------------------+--------------------+-----------+--------------------+----------------+------+-----------------------+
|MIN_YEARS_EXPERIENCE|MAX_YEARS_EXPERIENCE|SALARY_FROM|            MSA_NAME|REMOTE_TYPE_NAME|SALARY|MIN_YEARS_EXPERIENCE_SQ|
+--------------------+--------------------+-----------+--------------------+----------------+------+-----------------------+
|                   2|                   2|      79500|New York-Newark-J...|          Onsite| 92962|                    4.0|
|                   2|                   2|      75026|         Jackson, MS|          Onsite| 75026|                    4.0|
| 

...

Training set size: 2,574 rows


Feature matrix shape: (2574, 216)
Label vector shape: (2574,)



Residual Standard Error: $30,006.80
Degrees of Freedom: 2357
❌ Error calculating statistics: Singular matrix
   This may happen with singular matrices or perfect multicollinearity.




=== COEFFICIENT ANALYSIS TABLE ===
             Feature   Coefficient
MIN_YEARS_EXPERIENCE   4346.702719
MAX_YEARS_EXPERIENCE   4346.702719
               MSA_0  12170.474476
               MSA_1   3875.848099
               MSA_2  -2216.381832
               MSA_3    263.206998
               MSA_4  15502.361770
               MSA_5   4670.381095
               MSA_6   1336.867872
               MSA_7  -4322.782072
               MSA_8  -8193.486727
               MSA_9    145.396541
              MSA_10   6137.504086
              MSA_11   4558.735547
              MSA_12  10003.789801
              MSA_13   1349.100445
              MSA_14  34760.900322
              MSA_15   1979.252241
              MSA_16  -2376.652197
              MSA_17 -21038.614191
              MSA_18   9384.031296
              MSA_19  -4119.615087
              MSA_20   1783.074739
              MSA_21 -11785.320427
              MSA_22  -4338.026653
              MSA_23  -9262.264065
              MSA_2


--------------------------------------------------------------------------------

🎯 TEST SET PERFORMANCE:
--------------------------------------------------------------------------------
Test R²: 0.3617
Test RMSE: $26,948.87
Test MAE: $21,366.08

📊 TRAINING vs TEST COMPARISON:
--------------------------------------------------------------------------------
Training R²: 0.4205 | Test R²: 0.3617 | Difference: 0.0588
✓ Good generalization - acceptable overfitting

✓ MODEL TRAINING AND EVALUATION COMPLETED!


# Generalized Linear Regression Summary
The following statistical summary reveals that...
- Experience matters significantly: each additional year of minimum or maximum experience requirement adds approximately $4,347 to the predicted salary, holding other factors constant. 
- Location is also a significant factor, with dramatic geographic variation—jobs in Omaha pay over $105,000 more than the baseline (York-Hanover, PA), while positions in Weirton-Steubenville pay $70,796 less, creating a range of $175,844 across metropolitan areas. Tech hubs like San Jose ($34,761 premium), San Francisco ($15,502 premium), and Austin ($10,004 premium) command substantial premiums, while many mid-sized and smaller cities show negative coefficients. 
- Remote work arrangements also significantly impact compensation: onsite positions pay $21,651 more than hybrid roles, while fully remote positions pay $1,998 less than hybrid, suggesting employers may offer location-based compensation adjustments or that hybrid roles command a premium for flexibility without full remote work.
- There are several location coefficients of exactly $0, meaning those MSAs have identical salary expectations to the baseline after controlling for experience and work type.

In [5]:
import pandas as pd
import numpy as np

# ============================================================================
# DETAILED INTERPRETATION OF GLM COEFFICIENTS
# ============================================================================
print("\n" + "="*80)
print("DETAILED GENERALIZED LINEAR REGRESSION MODEL INTERPRETATION")
print("="*80 + "\n")

# Check if required variables exist
required_vars = ['lr_model', 'coefficients', 'pipeline_model', 'df_clean']
missing_vars = [var for var in required_vars if var not in dir()]

if missing_vars:
    print(f"❌ ERROR: Missing required variables: {missing_vars}")
    print("\n⚠️  This code must be run AFTER the model training code.")
    print("   Please run the Linear Regression training code first, then run this interpretation code.")
    print("\n   Required variables from training:")
    print("   - lr_model (trained model)")
    print("   - coefficients (model coefficients)")
    print("   - pipeline_model (fitted pipeline)")
    print("   - df_clean (cleaned dataframe)")
    print("   - stats_available (whether stats were calculated)")
    print("   - std_errors, t_values, p_values, ci_lower, ci_upper (if stats_available)")
    raise ValueError("Missing required variables from model training")

print("✓ All required variables found. Proceeding with interpretation...\n")

# ============================================================================
# PART 1: EXTRACT AND MAP FEATURE NAMES
# ============================================================================
print("="*80)
print("PART 1: FEATURE NAME MAPPING")
print("="*80 + "\n")

# Get the actual categorical mappings from the StringIndexer models
msa_model = pipeline_model.stages[0]  # msa_indexer
remote_model = pipeline_model.stages[1]  # remote_indexer

# Get the labels (original category names) in order of their indices
msa_labels = msa_model.labels
remote_labels = remote_model.labels

print(f"Number of MSA categories: {len(msa_labels)}")
print(f"Number of Remote Type categories: {len(remote_labels)}")

# Create detailed feature names with actual category labels
# OneHotEncoder with dropLast=True means we have n-1 features for n categories
# The dropped category becomes the reference/baseline category

feature_names_detailed = []

# Numerical features
feature_names_detailed.append('MIN_YEARS_EXPERIENCE')
feature_names_detailed.append('MAX_YEARS_EXPERIENCE')

# MSA (Metropolitan Statistical Area) - one-hot encoded
# dropLast=True means the last category is the baseline/reference
print(f"\nMSA Categories (Total: {len(msa_labels)}):")
for i, label in enumerate(msa_labels):
    print(f"  {i}: {label}")

baseline_msa = msa_labels[-1]  # Last one is dropped (baseline)
print(f"\n⭐ Baseline MSA (reference category): {baseline_msa}")
print(f"   All other MSA coefficients are relative to {baseline_msa}\n")

for i in range(len(msa_labels) - 1):  # All except the last one
    feature_names_detailed.append(f'MSA: {msa_labels[i]}')

# Remote Type - one-hot encoded
print(f"Remote Type Categories (Total: {len(remote_labels)}):")
for i, label in enumerate(remote_labels):
    print(f"  {i}: {label}")

baseline_remote = remote_labels[-1]  # Last one is dropped (baseline)
print(f"\n⭐ Baseline Remote Type (reference category): {baseline_remote}")
print(f"   All other Remote Type coefficients are relative to {baseline_remote}\n")

for i in range(len(remote_labels) - 1):  # All except the last one
    feature_names_detailed.append(f'Remote: {remote_labels[i]}')

print(f"Total features in model: {len(feature_names_detailed)}")
print(f"\nFeature list:")
for i, name in enumerate(feature_names_detailed):
    print(f"  {i+1}. {name}")

# ============================================================================
# PART 2: CREATE COMPREHENSIVE COEFFICIENT TABLE
# ============================================================================
print("\n" + "="*80)
print("PART 2: COMPREHENSIVE COEFFICIENT ANALYSIS")
print("="*80 + "\n")

# Recreate the coefficient DataFrame with detailed names
coef_data_detailed = []
for i, name in enumerate(feature_names_detailed):
    coef = float(coefficients[i])
    row_data = {
        'Feature': name,
        'Coefficient': coef,
        'Coef_Formatted': f'${coef:,.2f}'
    }
    
    if stats_available:
        se = float(std_errors[i])
        t = float(t_values[i])
        p = float(p_values[i])
        ci_low = float(ci_lower[i])
        ci_high = float(ci_upper[i])
        
        # Determine significance level
        if p < 0.001:
            sig_level = '***'
            sig_text = 'Highly Significant'
        elif p < 0.01:
            sig_level = '**'
            sig_text = 'Very Significant'
        elif p < 0.05:
            sig_level = '*'
            sig_text = 'Significant'
        elif p < 0.10:
            sig_level = '.'
            sig_text = 'Marginally Significant'
        else:
            sig_level = ''
            sig_text = 'Not Significant'
        
        row_data.update({
            'Std_Error': se,
            'T_Value': t,
            'P_Value': p,
            'CI_95_Lower': ci_low,
            'CI_95_Upper': ci_high,
            'Sig_Level': sig_level,
            'Significance': sig_text
        })
    
    coef_data_detailed.append(row_data)

coef_df_detailed = pd.DataFrame(coef_data_detailed)

# Display the full table
print("="*80)
print("FULL COEFFICIENT TABLE WITH DETAILED FEATURE NAMES")
print("="*80)
print("\nSignificance codes: '***' p<0.001, '**' p<0.01, '*' p<0.05, '.' p<0.10\n")
print(coef_df_detailed.to_string(index=False))

# ============================================================================
# PART 3: INTERPRET NUMERICAL FEATURES
# ============================================================================
print("\n" + "="*80)
print("PART 3: INTERPRETATION OF NUMERICAL FEATURES")
print("="*80 + "\n")

numerical_features = coef_df_detailed[coef_df_detailed['Feature'].isin(['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE'])]

for idx, row in numerical_features.iterrows():
    feature = row['Feature']
    coef = row['Coefficient']
    
    print(f"{'='*70}")
    print(f"Feature: {feature}")
    print(f"{'='*70}")
    print(f"Coefficient: ${coef:,.2f}")
    
    if stats_available:
        print(f"Standard Error: ${row['Std_Error']:,.2f}")
        print(f"T-Value: {row['T_Value']:.3f}")
        print(f"P-Value: {row['P_Value']:.4f} {row['Sig_Level']}")
        print(f"95% Confidence Interval: [${row['CI_95_Lower']:,.2f}, ${row['CI_95_Upper']:,.2f}]")
        print(f"Significance: {row['Significance']}")
    
    print(f"\n📊 INTERPRETATION:")
    if coef > 0:
        print(f"• For each additional year in {feature.replace('_', ' ').lower()},")
        print(f"  the predicted salary INCREASES by ${abs(coef):,.2f}, holding all other factors constant.")
    else:
        print(f"• For each additional year in {feature.replace('_', ' ').lower()},")
        print(f"  the predicted salary DECREASES by ${abs(coef):,.2f}, holding all other factors constant.")
    
    if stats_available:
        if row['P_Value'] < 0.05:
            print(f"• This effect is STATISTICALLY SIGNIFICANT (p = {row['P_Value']:.4f})")
            print(f"• We can be 95% confident the true effect is between ${row['CI_95_Lower']:,.2f} and ${row['CI_95_Upper']:,.2f}")
        else:
            print(f"• This effect is NOT statistically significant (p = {row['P_Value']:.4f})")
            print(f"• We cannot confidently say this feature affects salary")
    
    print()

# ============================================================================
# PART 4: INTERPRET CATEGORICAL FEATURES (MSA)
# ============================================================================
print("\n" + "="*80)
print("PART 4: INTERPRETATION OF MSA (LOCATION) EFFECTS")
print("="*80)
print(f"\n⭐ Baseline/Reference Category: {baseline_msa}")
print(f"   All coefficients below compare each MSA to {baseline_msa}\n")

msa_features = coef_df_detailed[coef_df_detailed['Feature'].str.startswith('MSA:')]

# Sort by coefficient value to see which locations pay most/least
msa_features_sorted = msa_features.sort_values('Coefficient', ascending=False)

print("="*70)
print("MSA SALARY EFFECTS (sorted by salary impact)")
print("="*70 + "\n")

for idx, row in msa_features_sorted.iterrows():
    location = row['Feature'].replace('MSA: ', '')
    coef = row['Coefficient']
    
    print(f"Location: {location}")
    print(f"  Coefficient: ${coef:,.2f}")
    
    if stats_available:
        print(f"  P-Value: {row['P_Value']:.4f} {row['Sig_Level']} ({row['Significance']})")
    
    if coef > 0:
        print(f"  💰 Jobs in {location} pay ${abs(coef):,.2f} MORE than {baseline_msa}")
    else:
        print(f"  💵 Jobs in {location} pay ${abs(coef):,.2f} LESS than {baseline_msa}")
    
    if stats_available:
        if row['P_Value'] < 0.05:
            print(f"  ✓ This difference IS statistically significant")
        else:
            print(f"  ✗ This difference is NOT statistically significant")
    
    print()

# Identify top and bottom paying locations
if len(msa_features_sorted) > 0:
    top_location = msa_features_sorted.iloc[0]
    bottom_location = msa_features_sorted.iloc[-1]
    
    print("="*70)
    print("KEY FINDINGS:")
    print("="*70)
    print(f"\n🏆 HIGHEST PAYING LOCATION (relative to {baseline_msa}):")
    print(f"   {top_location['Feature'].replace('MSA: ', '')}")
    print(f"   Premium: ${top_location['Coefficient']:,.2f}")
    if stats_available and top_location['P_Value'] < 0.05:
        print(f"   ✓ Statistically significant (p = {top_location['P_Value']:.4f})")
    
    print(f"\n📉 LOWEST PAYING LOCATION (relative to {baseline_msa}):")
    print(f"   {bottom_location['Feature'].replace('MSA: ', '')}")
    print(f"   Difference: ${bottom_location['Coefficient']:,.2f}")
    if stats_available and bottom_location['P_Value'] < 0.05:
        print(f"   ✓ Statistically significant (p = {bottom_location['P_Value']:.4f})")
    
    location_spread = top_location['Coefficient'] - bottom_location['Coefficient']
    print(f"\n📊 LOCATION SALARY SPREAD:")
    print(f"   Difference between highest and lowest paying locations: ${location_spread:,.2f}")

# ============================================================================
# PART 5: INTERPRET CATEGORICAL FEATURES (REMOTE TYPE)
# ============================================================================
print("\n" + "="*80)
print("PART 5: INTERPRETATION OF REMOTE WORK TYPE EFFECTS")
print("="*80)
print(f"\n⭐ Baseline/Reference Category: {baseline_remote}")
print(f"   All coefficients below compare each work type to {baseline_remote}\n")

remote_features = coef_df_detailed[coef_df_detailed['Feature'].str.startswith('Remote:')]

# Sort by coefficient value
remote_features_sorted = remote_features.sort_values('Coefficient', ascending=False)

print("="*70)
print("REMOTE WORK TYPE SALARY EFFECTS (sorted by salary impact)")
print("="*70 + "\n")

for idx, row in remote_features_sorted.iterrows():
    work_type = row['Feature'].replace('Remote: ', '')
    coef = row['Coefficient']
    
    print(f"Work Type: {work_type}")
    print(f"  Coefficient: ${coef:,.2f}")
    
    if stats_available:
        print(f"  P-Value: {row['P_Value']:.4f} {row['Sig_Level']} ({row['Significance']})")
    
    if coef > 0:
        print(f"  💰 {work_type} positions pay ${abs(coef):,.2f} MORE than {baseline_remote}")
    else:
        print(f"  💵 {work_type} positions pay ${abs(coef):,.2f} LESS than {baseline_remote}")
    
    if stats_available:
        if row['P_Value'] < 0.05:
            print(f"  ✓ This difference IS statistically significant")
        else:
            print(f"  ✗ This difference is NOT statistically significant")
    
    print()

# ============================================================================
# PART 6: OVERALL MODEL INSIGHTS
# ============================================================================
print("\n" + "="*80)
print("PART 6: OVERALL MODEL INSIGHTS AND RECOMMENDATIONS")
print("="*80 + "\n")

if stats_available:
    sig_features = coef_df_detailed[coef_df_detailed['P_Value'] < 0.05]
    highly_sig_features = coef_df_detailed[coef_df_detailed['P_Value'] < 0.001]
    
    print("📈 STATISTICAL SUMMARY:")
    print("="*70)
    print(f"Total features in model: {len(coef_df_detailed)}")
    print(f"Significant features (p < 0.05): {len(sig_features)} ({len(sig_features)/len(coef_df_detailed)*100:.1f}%)")
    print(f"Highly significant features (p < 0.001): {len(highly_sig_features)} ({len(highly_sig_features)/len(coef_df_detailed)*100:.1f}%)")
    
    print("\n🎯 MOST INFLUENTIAL FEATURES:")
    print("="*70)
    
    # Get features with largest absolute coefficients that are significant
    sig_features_abs = sig_features.copy()
    sig_features_abs['Abs_Coefficient'] = sig_features_abs['Coefficient'].abs()
    top_features = sig_features_abs.nlargest(5, 'Abs_Coefficient')
    
    print("\nTop 5 most impactful significant features:")
    for i, (idx, row) in enumerate(top_features.iterrows(), 1):
        print(f"\n{i}. {row['Feature']}")
        print(f"   Impact: ${row['Coefficient']:,.2f}")
        print(f"   Significance: {row['Significance']} (p = {row['P_Value']:.4f})")

print("\n" + "="*70)
print("💡 BUSINESS INSIGHTS:")
print("="*70)

print("\n1. EXPERIENCE FACTORS:")
if 'MIN_YEARS_EXPERIENCE' in coef_df_detailed['Feature'].values:
    min_exp_coef = coef_df_detailed[coef_df_detailed['Feature'] == 'MIN_YEARS_EXPERIENCE']['Coefficient'].values[0]
    if stats_available:
        min_exp_p = coef_df_detailed[coef_df_detailed['Feature'] == 'MIN_YEARS_EXPERIENCE']['P_Value'].values[0]
        if min_exp_p < 0.05:
            print(f"   • Minimum experience requirement significantly affects salary")
            print(f"   • Each additional year adds ~${min_exp_coef:,.2f} to salary")

if 'MAX_YEARS_EXPERIENCE' in coef_df_detailed['Feature'].values:
    max_exp_coef = coef_df_detailed[coef_df_detailed['Feature'] == 'MAX_YEARS_EXPERIENCE']['Coefficient'].values[0]
    if stats_available:
        max_exp_p = coef_df_detailed[coef_df_detailed['Feature'] == 'MAX_YEARS_EXPERIENCE']['P_Value'].values[0]
        if max_exp_p < 0.05:
            print(f"   • Maximum experience requirement significantly affects salary")
            print(f"   • Each additional year adds ~${max_exp_coef:,.2f} to salary")

print("\n2. LOCATION FACTORS:")
print(f"   • Location matters! Different MSAs show varying salary levels")
print(f"   • Baseline location: {baseline_msa}")
if len(msa_features_sorted) > 0:
    print(f"   • Location premium ranges from ${msa_features_sorted['Coefficient'].min():,.2f} to ${msa_features_sorted['Coefficient'].max():,.2f}")

print("\n3. REMOTE WORK FACTORS:")
print(f"   • Baseline work arrangement: {baseline_remote}")
if len(remote_features_sorted) > 0:
    print(f"   • Remote work type affects salary differently")
    print(f"   • Premium/discount ranges from ${remote_features_sorted['Coefficient'].min():,.2f} to ${remote_features_sorted['Coefficient'].max():,.2f}")

print("\n" + "="*70)
print("⚠️  IMPORTANT NOTES:")
print("="*70)
print("\n1. INTERPRETATION OF CATEGORICAL VARIABLES:")
print("   • One-hot encoding with dropLast=True creates reference categories")
print(f"   • MSA baseline: {baseline_msa}")
print(f"   • Remote Type baseline: {baseline_remote}")
print("   • All coefficients are relative to these baselines")

print("\n2. COEFFICIENT INTERPRETATION:")
print("   • Positive coefficient = higher salary than baseline")
print("   • Negative coefficient = lower salary than baseline")
print("   • Magnitude shows the dollar amount difference")

print("\n3. STATISTICAL SIGNIFICANCE:")
print("   • P-value < 0.05 means the effect is unlikely due to chance")
print("   • Confidence intervals show the range of plausible values")
print("   • T-values measure how many standard errors the coefficient is from zero")

print("\n4. DATA LEAKAGE RESOLUTION:")
print("   • SALARY_FROM was excluded from features to prevent data leakage")
print("   • This ensures the model uses only information available before knowing salary")
print("   • Results now reflect realistic prediction scenarios")

print("\n" + "="*80)
print("✓ DETAILED INTERPRETATION COMPLETED!")
print("="*80)


DETAILED GENERALIZED LINEAR REGRESSION MODEL INTERPRETATION

✓ All required variables found. Proceeding with interpretation...

PART 1: FEATURE NAME MAPPING

Number of MSA categories: 211
Number of Remote Type categories: 3

MSA Categories (Total: 211):
  0: New York-Newark-Jersey City, NY-NJ-PA
  1: Washington-Arlington-Alexandria, DC-VA-MD-WV
  2: Los Angeles-Long Beach-Anaheim, CA
  3: Dallas-Fort Worth-Arlington, TX
  4: San Francisco-Oakland-Berkeley, CA
  5: Chicago-Naperville-Elgin, IL-IN-WI
  6: Boston-Cambridge-Newton, MA-NH
  7: Denver-Aurora-Lakewood, CO
  8: Philadelphia-Camden-Wilmington, PA-NJ-DE-MD
  9: Tampa-St. Petersburg-Clearwater, FL
  10: Seattle-Tacoma-Bellevue, WA
  11: Phoenix-Mesa-Chandler, AZ
  12: Austin-Round Rock-Georgetown, TX
  13: Atlanta-Sandy Springs-Alpharetta, GA
  14: San Jose-Sunnyvale-Santa Clara, CA
  15: Houston-The Woodlands-Sugar Land, TX
  16: Baltimore-Columbia-Towson, MD
  17: Jackson, MS
  18: Columbus, OH
  19: Miami-Fort Lauderdale-Pomp

# Polynoimal Linear Regression & Summary
The following coefficients reveal that...
- MIN_YEARS_EXPERIENCE_SQ coefficient = -$487.46 (negative and significant): This indicates that each additional year of experience adds progressively less value to salary. For example, going from 0 to 1 years adds more salary than going from 10 to 11 years
-- MIN_YEARS_EXPERIENCE = +$6,677.47: the first year of experience adds $6,677.47
- MAX years experience has the same coefficients, suggesting the same effects or possibly duplication in the data
Location coefficients:
- San Jose = +$33,059: Highest tech hub premium relative to baseline
- San Francisco = +$15,768: Strong tech market premium
- New York = +$12,077: Major metro premium
- Austin = +$10,464: Emerging tech hub
- No meaningul insights for work arrangement

In [6]:

# ============================================================================
# POLYNOMIAL LINEAR REGRESSION MODEL TRAINING
# ============================================================================
print("\n" + "="*80)
print("POLYNOMIAL LINEAR REGRESSION MODEL")
print("="*80)

# CRITICAL ISSUE RESOLUTION FOR POLYNOMIAL FEATURES:
# The 'features_poly' column includes BOTH MIN_YEARS_EXPERIENCE_SQ AND SALARY_FROM
# 
# TWO MAJOR PROBLEMS:
# 1. DATA LEAKAGE: SALARY_FROM is part of the salary range (same as target SALARY)
#    - This violates ML independence assumptions
#    - Creates unrealistic model that can't be used for real predictions
# 
# 2. PERFECT MULTICOLLINEARITY: MIN_YEARS_EXPERIENCE and MIN_YEARS_EXPERIENCE_SQ
#    - These are perfectly correlated (one is just the square of the other)
#    - Can cause numerical instability in coefficient estimation
#    - Makes interpretation difficult
#
# SOLUTION: Create new polynomial features WITHOUT SALARY_FROM

print("\n⚠️  IDENTIFYING THE KEY ISSUES:")
print("="*80)
print("ISSUE 1: DATA LEAKAGE")
print("  • 'features_poly' includes SALARY_FROM")
print("  • SALARY_FROM is derived from the same job posting as SALARY (target)")
print("  • This creates unrealistic model performance\n")

print("ISSUE 2: MULTICOLLINEARITY")
print("  • Including both MIN_YEARS_EXPERIENCE and MIN_YEARS_EXPERIENCE_SQ")
print("  • High correlation between a variable and its square")
print("  • Can cause unstable coefficient estimates\n")

print("SOLUTION:")
print("  ✓ Remove SALARY_FROM to prevent data leakage")
print("  ✓ Keep polynomial term for legitimate non-linear relationship modeling")
print("  ✓ Multicollinearity with polynomial terms is ACCEPTABLE when modeling")
print("    non-linear relationships (this is standard practice)")
print("="*80 + "\n")

# Create polynomial features WITHOUT SALARY_FROM
poly_feature_cols_clean = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ',
                           'MAX_YEARS_EXPERIENCE',
                           'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

poly_assembler_clean = VectorAssembler(inputCols=poly_feature_cols_clean, 
                                       outputCol='features_poly_clean',
                                       handleInvalid='keep')

# Transform data with clean polynomial features
df_train_poly = poly_assembler_clean.transform(train_data)
df_test_poly = poly_assembler_clean.transform(test_data)

print("✓ Created 'features_poly_clean' column WITHOUT SALARY_FROM")
print(f"  Features included: {poly_feature_cols_clean}\n")

# Initialize Polynomial Linear Regression model
lr_poly = LinearRegression(
    featuresCol='features_poly_clean',
    labelCol='SALARY',
    maxIter=100,
    regParam=0.0,
    elasticNetParam=0.0,
    standardization=True
)

# Train the model
print("Training Polynomial Linear Regression model...")
lr_poly_model = lr_poly.fit(df_train_poly)
print("✓ Polynomial model training completed!\n")

# ============================================================================
# MODEL EVALUATION
# ============================================================================
print("="*80)
print("POLYNOMIAL MODEL EVALUATION")
print("="*80 + "\n")

# Make predictions on test data
predictions_poly = lr_poly_model.transform(df_test_poly)

# Display sample predictions
print("=== SAMPLE PREDICTIONS (POLYNOMIAL MODEL) ===")
predictions_poly.select('SALARY', 'prediction', 'MIN_YEARS_EXPERIENCE', 
                        'MAX_YEARS_EXPERIENCE', 'MSA_NAME', 'REMOTE_TYPE_NAME').show(10)

# ============================================================================
# EXTRACT MODEL COEFFICIENTS AND CALCULATE STATISTICS
# ============================================================================
print("\n" + "="*80)
print("POLYNOMIAL MODEL COEFFICIENTS AND STATISTICS")
print("="*80 + "\n")

# Get model summary
summary_poly = lr_poly_model.summary

# Extract basic metrics
intercept_poly = lr_poly_model.intercept
coefficients_poly = lr_poly_model.coefficients
r2_poly = summary_poly.r2
rmse_poly = summary_poly.rootMeanSquaredError
mae_poly = summary_poly.meanAbsoluteError

print(f"Intercept: ${intercept_poly:,.2f}")
print(f"R² (R-squared): {r2_poly:.4f}")
print(f"RMSE (Root Mean Squared Error): ${rmse_poly:,.2f}")
print(f"MAE (Mean Absolute Error): ${mae_poly:,.2f}")

# ============================================================================
# MANUAL CALCULATION OF COEFFICIENT STATISTICS
# ============================================================================
print("\n" + "="*80)
print("CALCULATING POLYNOMIAL MODEL COEFFICIENT STATISTICS")
print("="*80 + "\n")

print("Extracting feature matrix and target values from training data...")

# Collect training data for manual statistics calculation
train_count_poly = df_train_poly.count()
print(f"Training set size: {train_count_poly:,} rows")

if train_count_poly > 100000:
    print("⚠️  Warning: Large dataset. Manual statistics calculation may be slow.")
    print("   Consider using a sample for coefficient statistics.\n")

# Extract features and labels
train_features_poly = np.array(df_train_poly.select('features_poly_clean').rdd.map(lambda row: row[0].toArray()).collect())
train_labels_poly = np.array(df_train_poly.select('SALARY').rdd.map(lambda row: row[0]).collect())

print(f"Feature matrix shape: {train_features_poly.shape}")
print(f"Label vector shape: {train_labels_poly.shape}")

# Get predictions on training data for residuals
train_predictions_poly = lr_poly_model.transform(df_train_poly)
train_pred_values_poly = np.array(train_predictions_poly.select('prediction').rdd.map(lambda row: row[0]).collect())

# Calculate residuals
residuals_poly = train_labels_poly - train_pred_values_poly
n_poly = len(train_labels_poly)
k_poly = train_features_poly.shape[1]  # number of features
df_residual_poly = n_poly - k_poly - 1  # degrees of freedom

# Calculate residual standard error
rse_poly = np.sqrt(np.sum(residuals_poly**2) / df_residual_poly)

print(f"\nResidual Standard Error: ${rse_poly:,.2f}")
print(f"Degrees of Freedom: {df_residual_poly}")

# Calculate variance-covariance matrix
try:
    X_poly = train_features_poly
    XtX_poly = np.dot(X_poly.T, X_poly)
    XtX_inv_poly = np.linalg.inv(XtX_poly)
    
    # Variance-covariance matrix
    var_covar_matrix_poly = (rse_poly**2) * XtX_inv_poly
    
    # Standard errors are square roots of diagonal elements
    std_errors_poly = np.sqrt(np.diag(var_covar_matrix_poly))
    
    # Calculate t-values
    coef_array_poly = np.array(coefficients_poly.toArray())
    t_values_poly = coef_array_poly / std_errors_poly
    
    # Calculate p-values (two-tailed test)
    p_values_poly = 2 * (1 - scipy_stats.t.cdf(np.abs(t_values_poly), df_residual_poly))
    
    # Calculate 95% confidence intervals
    t_critical_poly = scipy_stats.t.ppf(0.975, df_residual_poly)
    ci_lower_poly = coef_array_poly - t_critical_poly * std_errors_poly
    ci_upper_poly = coef_array_poly + t_critical_poly * std_errors_poly
    
    stats_available_poly = True
    print("✓ Coefficient statistics calculated successfully!\n")
    
except np.linalg.LinAlgError as e:
    print(f"❌ Error calculating statistics: {e}")
    print("   This may happen with singular matrices or perfect multicollinearity.\n")
    stats_available_poly = False
    std_errors_poly = [None] * len(coefficients_poly)
    t_values_poly = [None] * len(coefficients_poly)
    p_values_poly = [None] * len(coefficients_poly)
    ci_lower_poly = [None] * len(coefficients_poly)
    ci_upper_poly = [None] * len(coefficients_poly)

# ============================================================================
# CREATE COEFFICIENT TABLE WITH FEATURE NAMES
# ============================================================================

# Create feature names for polynomial model
num_msa_categories = df_clean.select('MSA_NAME').distinct().count() - 1
num_remote_categories = df_clean.select('REMOTE_TYPE_NAME').distinct().count() - 1

feature_names_poly = ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 'MAX_YEARS_EXPERIENCE']
feature_names_poly += [f'MSA_{i}' for i in range(num_msa_categories)]
feature_names_poly += [f'REMOTE_{i}' for i in range(num_remote_categories)]

# Create DataFrame for coefficient analysis
coef_data_poly = []
for i, (name, coef) in enumerate(zip(feature_names_poly, coefficients_poly)):
    row_data = {
        'Feature': name,
        'Coefficient': float(coef)
    }
    
    if stats_available_poly:
        row_data.update({
            'Std_Error': float(std_errors_poly[i]),
            'T_Value': float(t_values_poly[i]),
            'P_Value': float(p_values_poly[i]),
            'CI_Lower': float(ci_lower_poly[i]),
            'CI_Upper': float(ci_upper_poly[i]),
            'Significant': '***' if p_values_poly[i] < 0.001 else '**' if p_values_poly[i] < 0.01 else '*' if p_values_poly[i] < 0.05 else 'No'
        })
    
    coef_data_poly.append(row_data)

# Convert to Pandas for better display
coef_df_poly = pd.DataFrame(coef_data_poly)

print("\n=== POLYNOMIAL MODEL COEFFICIENT ANALYSIS TABLE (TOP 20 FEATURES) ===")
if stats_available_poly:
    print(coef_df_poly.head(20).to_string(index=False))
else:
    print(coef_df_poly.head(20).to_string(index=False))
    print("\nNote: Statistical tests not available")

# ============================================================================
# INTERPRET RESULTS
# ============================================================================
print("\n" + "="*80)
print("POLYNOMIAL MODEL INTERPRETATION")
print("="*80 + "\n")

print("📊 POLYNOMIAL FEATURES INTERPRETATION:")
print("-" * 80)

if stats_available_poly:
    # Interpret the first 3 numerical features (including polynomial term)
    for i in range(min(3, len(coef_df_poly))):
        row = coef_df_poly.iloc[i]
        name = row['Feature']
        coef = row['Coefficient']
        p_val = row['P_Value']
        sig = row['Significant']
        
        print(f"\n{name}:")
        print(f"  • Coefficient: ${coef:,.2f} {sig}")
        
        if 'SQ' in name:
            print(f"  • Interpretation: This is the SQUARED term for MIN_YEARS_EXPERIENCE")
            if coef > 0:
                print(f"  • Effect: Creates an ACCELERATING (convex) relationship")
                print(f"  • Meaning: Each additional year of experience has INCREASING marginal value")
            else:
                print(f"  • Effect: Creates a DECELERATING (concave) relationship")
                print(f"  • Meaning: Each additional year of experience has DECREASING marginal value")
        else:
            print(f"  • Interpretation: Linear effect on salary")
            print(f"    Each additional year {'increases' if coef > 0 else 'decreases'} salary by ${abs(coef):,.2f}")
        
        print(f"  • Statistical Significance: {sig} (p={p_val:.4f})")
        print(f"  • 95% CI: [${row['CI_Lower']:,.2f}, ${row['CI_Upper']:,.2f}]")
else:
    for i in range(min(3, len(coef_df_poly))):
        row = coef_df_poly.iloc[i]
        name = row['Feature']
        coef = row['Coefficient']
        
        print(f"\n{name}:")
        print(f"  • Coefficient: ${coef:,.2f}")
        
        if 'SQ' in name:
            print(f"  • This is the SQUARED term - captures non-linear relationship")
        else:
            print(f"  • Linear effect on salary")

print("\n" + "-" * 80)
print("\n📈 POLYNOMIAL MODEL PERFORMANCE METRICS:")
print("-" * 80)

print(f"\n1. R² (R-squared) = {r2_poly:.4f}")
print(f"   • Interpretation: The polynomial model explains {r2_poly*100:.2f}% of variance in salary")
print(f"   • Comparison to linear model: R² = {r2:.4f} (linear) vs {r2_poly:.4f} (polynomial)")
r2_improvement = (r2_poly - r2) * 100
if r2_improvement > 0:
    print(f"   • Improvement: +{r2_improvement:.2f} percentage points")
    if r2_improvement > 2:
        print(f"   • Assessment: Polynomial terms provide MEANINGFUL improvement")
    else:
        print(f"   • Assessment: Minimal improvement - polynomial may not be necessary")
else:
    print(f"   • Assessment: Polynomial model performs WORSE - overfitting likely")

print(f"\n2. RMSE (Root Mean Squared Error) = ${rmse_poly:,.2f}")
print(f"   • Interpretation: Average prediction error is ${rmse_poly:,.2f}")
print(f"   • Comparison to linear model: ${rmse:,.2f} (linear) vs ${rmse_poly:,.2f} (polynomial)")
rmse_improvement = ((rmse - rmse_poly) / rmse) * 100
if rmse_improvement > 0:
    print(f"   • Improvement: {rmse_improvement:.2f}% reduction in error")
else:
    print(f"   • Degradation: {abs(rmse_improvement):.2f}% increase in error")

print(f"\n3. MAE (Mean Absolute Error) = ${mae_poly:,.2f}")
print(f"   • Interpretation: Average absolute prediction error is ${mae_poly:,.2f}")
print(f"   • Comparison to linear model: ${mae:,.2f} (linear) vs ${mae_poly:,.2f} (polynomial)")
mae_improvement = ((mae - mae_poly) / mae) * 100
if mae_improvement > 0:
    print(f"   • Improvement: {mae_improvement:.2f}% reduction in error")
else:
    print(f"   • Degradation: {abs(mae_improvement):.2f}% increase in error")

# Calculate test set performance
evaluator_r2_poly = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")
evaluator_rmse_poly = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
evaluator_mae_poly = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="mae")

test_r2_poly = evaluator_r2_poly.evaluate(predictions_poly)
test_rmse_poly = evaluator_rmse_poly.evaluate(predictions_poly)
test_mae_poly = evaluator_mae_poly.evaluate(predictions_poly)

print("\n" + "-" * 80)
print("\n🎯 TEST SET PERFORMANCE (POLYNOMIAL MODEL):")
print("-" * 80)
print(f"Test R²: {test_r2_poly:.4f}")
print(f"Test RMSE: ${test_rmse_poly:,.2f}")
print(f"Test MAE: ${test_mae_poly:,.2f}")

# Compare training vs test performance
print("\n📊 TRAINING vs TEST COMPARISON (POLYNOMIAL MODEL):")
print("-" * 80)
print(f"Training R²: {r2_poly:.4f} | Test R²: {test_r2_poly:.4f} | Difference: {abs(r2_poly-test_r2_poly):.4f}")
if abs(r2_poly - test_r2_poly) < 0.05:
    print("✓ Excellent generalization - minimal overfitting")
elif abs(r2_poly - test_r2_poly) < 0.10:
    print("✓ Good generalization - acceptable overfitting")
elif abs(r2_poly - test_r2_poly) < 0.15:
    print("⚠ Moderate overfitting detected - consider regularization")
else:
    print("❌ Significant overfitting - polynomial model may not generalize well")

# ============================================================================
# MODEL COMPARISON: LINEAR vs POLYNOMIAL
# ============================================================================
print("\n" + "="*80)
print("MODEL COMPARISON: LINEAR vs POLYNOMIAL")
print("="*80 + "\n")

comparison_data = {
    'Metric': ['R² (Training)', 'R² (Test)', 'RMSE (Training)', 'RMSE (Test)', 
               'MAE (Training)', 'MAE (Test)'],
    'Linear Model': [f'{r2:.4f}', f'{test_r2:.4f}', f'${rmse:,.2f}', f'${test_rmse:,.2f}',
                     f'${mae:,.2f}', f'${test_mae:,.2f}'],
    'Polynomial Model': [f'{r2_poly:.4f}', f'{test_r2_poly:.4f}', f'${rmse_poly:,.2f}', 
                         f'${test_rmse_poly:,.2f}', f'${mae_poly:,.2f}', f'${test_mae_poly:,.2f}']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

if stats_available_poly:
    print("\n" + "-" * 80)
    print("\n🔍 STATISTICAL INSIGHTS (POLYNOMIAL MODEL):")
    print("-" * 80)
    sig_features_poly = coef_df_poly[coef_df_poly['Significant'] != 'No'] if 'Significant' in coef_df_poly.columns else pd.DataFrame()
    if len(sig_features_poly) > 0:
        print(f"Number of significant features (p < 0.05): {len(sig_features_poly)}")
        print(f"Total features: {len(coef_df_poly)}")
        print(f"Percentage significant: {len(sig_features_poly)/len(coef_df_poly)*100:.1f}%")
    
    adj_r2_poly = 1 - (1-r2_poly)*(n_poly-1)/(n_poly-k_poly-1)
    print(f"\nAdjusted R² (Polynomial): {adj_r2_poly:.4f}")
    print(f"  • Accounts for number of predictors")
    print(f"  • Penalizes model complexity")

print("\n" + "-" * 80)
print("\n💡 KEY INSIGHTS:")
print("-" * 80)

print("\n1. POLYNOMIAL TERM EFFECT:")
print(f"   • MIN_YEARS_EXPERIENCE_SQ coefficient: ${coef_df_poly.iloc[1]['Coefficient']:,.2f}")
if coef_df_poly.iloc[1]['Coefficient'] > 0:
    print(f"   • Positive squared term indicates ACCELERATING returns to experience")
    print(f"   • Each additional year of experience adds MORE value than the previous year")
else:
    print(f"   • Negative squared term indicates DIMINISHING returns to experience")
    print(f"   • Each additional year of experience adds LESS value than the previous year")

print("\n2. MODEL SELECTION:")
test_r2_diff = test_r2_poly - test_r2
if test_r2_diff > 0.02:
    print(f"   ✓ POLYNOMIAL MODEL RECOMMENDED")
    print(f"   • Test R² improved by {test_r2_diff:.4f}")
    print(f"   • Better captures non-linear relationships")
elif test_r2_diff > -0.01:
    print(f"   ≈ MODELS PERFORM SIMILARLY")
    print(f"   • Consider LINEAR MODEL for simplicity")
    print(f"   • Polynomial adds complexity without substantial benefit")
else:
    print(f"   ✓ LINEAR MODEL RECOMMENDED")
    print(f"   • Polynomial model shows overfitting")
    print(f"   • Simpler linear model generalizes better")

print("\n3. DATA LEAKAGE RESOLUTION:")
print("   ✓ SALARY_FROM excluded from both models")
print("   ✓ Models use only pre-salary information")
print("   ✓ Results reflect realistic prediction scenarios")

print("\n" + "="*80)
print("✓ POLYNOMIAL MODEL TRAINING AND EVALUATION COMPLETED!")
print("="*80)


POLYNOMIAL LINEAR REGRESSION MODEL

⚠️  IDENTIFYING THE KEY ISSUES:
ISSUE 1: DATA LEAKAGE
  • 'features_poly' includes SALARY_FROM
  • SALARY_FROM is derived from the same job posting as SALARY (target)
  • This creates unrealistic model performance

ISSUE 2: MULTICOLLINEARITY
  • Including both MIN_YEARS_EXPERIENCE and MIN_YEARS_EXPERIENCE_SQ
  • High correlation between a variable and its square
  • Can cause unstable coefficient estimates

SOLUTION:
  ✓ Remove SALARY_FROM to prevent data leakage
  ✓ Keep polynomial term for legitimate non-linear relationship modeling
  ✓ Multicollinearity with polynomial terms is ACCEPTABLE when modeling
    non-linear relationships (this is standard practice)



✓ Created 'features_poly_clean' column WITHOUT SALARY_FROM
  Features included: ['MIN_YEARS_EXPERIENCE', 'MIN_YEARS_EXPERIENCE_SQ', 'MAX_YEARS_EXPERIENCE', 'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

Training Polynomial Linear Regression model...


25/10/08 23:15:18 WARN Instrumentation: [f6bb1389] regParam is zero, which might cause numerical instability and overfitting.
25/10/08 23:15:23 WARN Instrumentation: [f6bb1389] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


✓ Polynomial model training completed!

POLYNOMIAL MODEL EVALUATION

=== SAMPLE PREDICTIONS (POLYNOMIAL MODEL) ===


+------+-----------------+--------------------+--------------------+--------------------+----------------+
|SALARY|       prediction|MIN_YEARS_EXPERIENCE|MAX_YEARS_EXPERIENCE|            MSA_NAME|REMOTE_TYPE_NAME|
+------+-----------------+--------------------+--------------------+--------------------+----------------+
| 49547|48293.28752570613|                   0|                   0|Riverside-San Ber...|          Onsite|
| 41600|70945.63330280867|                   0|                   0|    Jacksonville, FL|          Onsite|
| 66500| 69672.3751949212|                   0|                   0|Houston-The Woodl...|          Onsite|
| 48880|63514.95910589581|                   0|                   0|Denver-Aurora-Lak...|          Onsite|
| 50960|71940.14443706104|                   0|                   0|Chicago-Napervill...|          Onsite|
| 61328|68048.23462625974|                   0|                   0|Dallas-Fort Worth...|          Onsite|
| 48922|64574.17597306847|           

Training set size: 2,574 rows


Feature matrix shape: (2574, 217)
Label vector shape: (2574,)



Residual Standard Error: $29,792.93
Degrees of Freedom: 2356
❌ Error calculating statistics: Singular matrix
   This may happen with singular matrices or perfect multicollinearity.




=== POLYNOMIAL MODEL COEFFICIENT ANALYSIS TABLE (TOP 20 FEATURES) ===
                Feature  Coefficient
   MIN_YEARS_EXPERIENCE  6677.465980
MIN_YEARS_EXPERIENCE_SQ  -487.464719
   MAX_YEARS_EXPERIENCE  6677.465980
                  MSA_0 12077.196478
                  MSA_1  5748.537947
                  MSA_2 -2379.658351
                  MSA_3   588.630858
                  MSA_4 15768.413900
                  MSA_5  4480.540669
                  MSA_6  1024.538592
                  MSA_7 -3944.644662
                  MSA_8 -8198.378398
                  MSA_9 -1001.453955
                 MSA_10  5793.802321
                 MSA_11  4157.992914
                 MSA_12 10463.939075
                 MSA_13  1796.482977
                 MSA_14 33058.888969
                 MSA_15  2212.771427
                 MSA_16 -1550.457717

Note: Statistical tests not available

POLYNOMIAL MODEL INTERPRETATION

📊 POLYNOMIAL FEATURES INTERPRETATION:
-----------------------------------------


--------------------------------------------------------------------------------

🎯 TEST SET PERFORMANCE (POLYNOMIAL MODEL):
--------------------------------------------------------------------------------
Test R²: 0.3723
Test RMSE: $26,723.22
Test MAE: $21,256.43

📊 TRAINING vs TEST COMPARISON (POLYNOMIAL MODEL):
--------------------------------------------------------------------------------
Training R²: 0.4290 | Test R²: 0.3723 | Difference: 0.0567
✓ Good generalization - acceptable overfitting

MODEL COMPARISON: LINEAR vs POLYNOMIAL

         Metric Linear Model Polynomial Model
  R² (Training)       0.4205           0.4290
      R² (Test)       0.3617           0.3723
RMSE (Training)   $28,714.10       $28,503.40
    RMSE (Test)   $26,948.87       $26,723.22
 MAE (Training)   $21,077.77       $21,028.48
     MAE (Test)   $21,366.08       $21,256.43

--------------------------------------------------------------------------------

💡 KEY INSIGHTS:
----------------------------------

# Random Forest Regressor

In [7]:


# ============================================================================
# RANDOM FOREST REGRESSOR MODEL TRAINING
# ============================================================================
print("\n" + "="*80)
print("RANDOM FOREST REGRESSOR MODEL")
print("="*80)

# CRITICAL ISSUE: DATA LEAKAGE (Same as before)
# The 'features' column includes SALARY_FROM which creates data leakage
# We need to use 'features_clean' instead (already created in linear model)

print("\n⚠️  DATA LEAKAGE ISSUE:")
print("="*80)
print("  • Original 'features' column includes SALARY_FROM")
print("  • SALARY_FROM is part of the same salary range as target (SALARY)")
print("  • Using 'features_clean' instead (without SALARY_FROM)")
print("="*80 + "\n")

# Verify features_clean exists or create it
feature_cols_clean = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE',
                      'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

assembler_clean = VectorAssembler(inputCols=feature_cols_clean, 
                                  outputCol='features_clean',
                                  handleInvalid='keep')

# Transform data with clean features (if not already done)
if 'features_clean' not in df_train.columns:
    df_train = assembler_clean.transform(train_data)
    df_test = assembler_clean.transform(test_data)

print("✓ Using 'features_clean' column WITHOUT SALARY_FROM")
print(f"  Features included: {feature_cols_clean}\n")

# ============================================================================
# HYPERPARAMETER SELECTION
# ============================================================================
print("="*80)
print("HYPERPARAMETER SELECTION")
print("="*80 + "\n")

# Key insight: numTrees and maxDepth are inversely proportional
# More trees → can use shallower trees (less overfitting per tree)
# Fewer trees → need deeper trees (capture more complexity per tree)

# Strategy: Use moderate number of trees with moderate depth for balance
NUM_TREES = 200  # Mid-range: good balance of performance and training time
MAX_DEPTH = 7    # Mid-range: captures complexity without severe overfitting

print("📊 HYPERPARAMETER CHOICES:")
print("-" * 80)
print(f"Number of Trees (numTrees): {NUM_TREES}")
print(f"  • Range: 100-500 trees")
print(f"  • Rationale: 200 trees provides good ensemble diversity")
print(f"  • More trees = more stable predictions, longer training time")
print(f"  • Diminishing returns typically after 200-300 trees\n")

print(f"Maximum Depth (maxDepth): {MAX_DEPTH}")
print(f"  • Range: 4-10 levels")
print(f"  • Rationale: Depth 7 balances complexity and generalization")
print(f"  • Deeper trees = more complex patterns, higher overfitting risk")
print(f"  • Shallower trees = simpler patterns, potential underfitting\n")

print("🔄 INVERSE RELATIONSHIP:")
print("-" * 80)
print("  • High trees (400-500) → Use shallow depth (4-5)")
print("  •   → Many simple trees average out noise")
print("  • Low trees (100-150) → Use deeper depth (8-10)")
print("  •   → Fewer trees need more complexity each")
print("  • BALANCED APPROACH (chosen): 200 trees × depth 7")
print("  •   → Moderate ensemble with moderate complexity per tree")
print("="*80 + "\n")

# ============================================================================
# INITIALIZE AND TRAIN RANDOM FOREST MODEL
# ============================================================================
print("="*80)
print("TRAINING RANDOM FOREST MODEL")
print("="*80 + "\n")

# Initialize Random Forest Regressor
rf = RandomForestRegressor(
    featuresCol='features_clean',
    labelCol='SALARY',
    numTrees=NUM_TREES,
    maxDepth=MAX_DEPTH,
    minInstancesPerNode=1,  # Minimum samples required at leaf node
    subsamplingRate=1.0,     # Use 100% of data for each tree (bagging)
    seed=42,                 # For reproducibility
    maxBins=32              # For handling categorical variables
)

# Train the model
print(f"Training Random Forest with {NUM_TREES} trees and max depth {MAX_DEPTH}...")
print("This may take a few moments...\n")

rf_model = rf.fit(df_train)

print("✓ Random Forest model training completed!\n")

# ============================================================================
# MODEL EVALUATION
# ============================================================================
print("="*80)
print("RANDOM FOREST MODEL EVALUATION")
print("="*80 + "\n")

# Make predictions on training data
train_predictions_rf = rf_model.transform(df_train)

# Make predictions on test data
test_predictions_rf = rf_model.transform(df_test)

# Display sample predictions
print("=== SAMPLE PREDICTIONS (RANDOM FOREST) ===")
test_predictions_rf.select('SALARY', 'prediction', 'MIN_YEARS_EXPERIENCE', 
                           'MAX_YEARS_EXPERIENCE', 'MSA_NAME', 'REMOTE_TYPE_NAME').show(10)

# ============================================================================
# CALCULATE PERFORMANCE METRICS
# ============================================================================
print("\n" + "="*80)
print("PERFORMANCE METRICS")
print("="*80 + "\n")

# Initialize evaluators
evaluator_r2 = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")
evaluator_rmse = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="mae")

# Training metrics
train_r2_rf = evaluator_r2.evaluate(train_predictions_rf)
train_rmse_rf = evaluator_rmse.evaluate(train_predictions_rf)
train_mae_rf = evaluator_mae.evaluate(train_predictions_rf)

# Test metrics
test_r2_rf = evaluator_r2.evaluate(test_predictions_rf)
test_rmse_rf = evaluator_rmse.evaluate(test_predictions_rf)
test_mae_rf = evaluator_mae.evaluate(test_predictions_rf)

print("📊 TRAINING SET PERFORMANCE:")
print("-" * 80)
print(f"R² (R-squared): {train_r2_rf:.4f}")
print(f"  • Interpretation: Model explains {train_r2_rf*100:.2f}% of variance on training data")
print(f"RMSE: ${train_rmse_rf:,.2f}")
print(f"  • Average prediction error on training data")
print(f"MAE: ${train_mae_rf:,.2f}")
print(f"  • Average absolute error on training data")

print("\n🎯 TEST SET PERFORMANCE:")
print("-" * 80)
print(f"R² (R-squared): {test_r2_rf:.4f}")
print(f"  • Interpretation: Model explains {test_r2_rf*100:.2f}% of variance on test data")
print(f"RMSE: ${test_rmse_rf:,.2f}")
print(f"  • Average prediction error on test data")
print(f"MAE: ${test_mae_rf:,.2f}")
print(f"  • Average absolute error on test data")

# Check for overfitting
print("\n📈 OVERFITTING ANALYSIS:")
print("-" * 80)
r2_diff = train_r2_rf - test_r2_rf
print(f"Training R²: {train_r2_rf:.4f}")
print(f"Test R²: {test_r2_rf:.4f}")
print(f"Difference: {r2_diff:.4f}")

if r2_diff < 0.05:
    print("✓ Excellent generalization - minimal overfitting")
elif r2_diff < 0.10:
    print("✓ Good generalization - acceptable overfitting")
elif r2_diff < 0.20:
    print("⚠ Moderate overfitting - consider reducing maxDepth or increasing minInstancesPerNode")
else:
    print("❌ Significant overfitting - model memorizing training data")

# ============================================================================
# FEATURE IMPORTANCE
# ============================================================================
print("\n" + "="*80)
print("FEATURE IMPORTANCE ANALYSIS")
print("="*80 + "\n")

# Extract feature importances
feature_importances = rf_model.featureImportances.toArray()

# Create feature names (same as linear model)
num_msa_categories = df_clean.select('MSA_NAME').distinct().count() - 1
num_remote_categories = df_clean.select('REMOTE_TYPE_NAME').distinct().count() - 1

feature_names_rf = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE']
feature_names_rf += [f'MSA_{i}' for i in range(num_msa_categories)]
feature_names_rf += [f'REMOTE_{i}' for i in range(num_remote_categories)]

# Create DataFrame for feature importance
importance_data = []
for i, (name, importance) in enumerate(zip(feature_names_rf, feature_importances)):
    importance_data.append({
        'Feature': name,
        'Importance': float(importance),
        'Importance_Pct': float(importance) * 100
    })

importance_df = pd.DataFrame(importance_data)
importance_df = importance_df.sort_values('Importance', ascending=False)

print("=== TOP 20 MOST IMPORTANT FEATURES ===")
print(importance_df.head(20).to_string(index=False))

print("\n" + "-" * 80)
print("\n📊 FEATURE IMPORTANCE INTERPRETATION:")
print("-" * 80)

# Analyze top features
top_5 = importance_df.head(5)
total_top_5_importance = top_5['Importance_Pct'].sum()

print(f"\nTop 5 features account for {total_top_5_importance:.2f}% of total importance")
print("\nKey Insights:")
for idx, row in top_5.iterrows():
    feature = row['Feature']
    importance = row['Importance_Pct']
    print(f"  • {feature}: {importance:.2f}%")
    
    if 'EXPERIENCE' in feature:
        print(f"    → Experience is a critical predictor of salary")
    elif 'MSA' in feature:
        print(f"    → This location significantly impacts salary predictions")
    elif 'REMOTE' in feature:
        print(f"    → Work arrangement is an important factor")

# ============================================================================
# MODEL COMPARISON: LINEAR vs POLYNOMIAL vs RANDOM FOREST
# ============================================================================
print("\n" + "="*80)
print("MODEL COMPARISON: LINEAR vs POLYNOMIAL vs RANDOM FOREST")
print("="*80 + "\n")

comparison_data = {
    'Metric': ['R² (Training)', 'R² (Test)', 'RMSE (Training)', 'RMSE (Test)', 
               'MAE (Training)', 'MAE (Test)', 'Overfitting (R² diff)'],
    'Linear Model': [f'{r2:.4f}', f'{test_r2:.4f}', f'${rmse:,.2f}', f'${test_rmse:,.2f}',
                     f'${mae:,.2f}', f'${test_mae:,.2f}', f'{abs(r2-test_r2):.4f}'],
    'Polynomial Model': [f'{r2_poly:.4f}', f'{test_r2_poly:.4f}', f'${rmse_poly:,.2f}', 
                         f'${test_rmse_poly:,.2f}', f'${mae_poly:,.2f}', f'${test_mae_poly:,.2f}',
                         f'{abs(r2_poly-test_r2_poly):.4f}'],
    'Random Forest': [f'{train_r2_rf:.4f}', f'{test_r2_rf:.4f}', f'${train_rmse_rf:,.2f}',
                      f'${test_rmse_rf:,.2f}', f'${train_mae_rf:,.2f}', f'${test_mae_rf:,.2f}',
                      f'{r2_diff:.4f}']
}

comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False))

print("\n" + "-" * 80)
print("\n💡 KEY INSIGHTS AND RECOMMENDATIONS:")
print("-" * 80)

# Determine best model based on test R²
models = {
    'Linear': test_r2,
    'Polynomial': test_r2_poly,
    'Random Forest': test_r2_rf
}

best_model = max(models, key=models.get)
best_r2 = models[best_model]

print(f"\n🏆 BEST PERFORMING MODEL: {best_model}")
print(f"   Test R²: {best_r2:.4f} ({best_r2*100:.2f}% variance explained)")

print("\n📊 MODEL CHARACTERISTICS:")

print("\n1. LINEAR MODEL:")
print(f"   • Test R²: {test_r2:.4f}")
print(f"   • Pros: Simple, interpretable coefficients, fast training")
print(f"   • Cons: Assumes linear relationships, may miss complex patterns")
print(f"   • Best for: Understanding feature effects, baseline comparisons")

print("\n2. POLYNOMIAL MODEL:")
print(f"   • Test R²: {test_r2_poly:.4f}")
print(f"   • Pros: Captures non-linear experience effects")
print(f"   • Cons: Only minimal improvement over linear")
print(f"   • Best for: When diminishing returns hypothesis needs testing")

print("\n3. RANDOM FOREST MODEL:")
print(f"   • Test R²: {test_r2_rf:.4f}")
print(f"   • Pros: Captures complex non-linear patterns, no feature scaling needed")
print(f"   • Cons: Less interpretable, longer training time")
print(f"   • Best for: Maximum predictive accuracy, feature importance analysis")

print("\n🎯 FINAL RECOMMENDATION:")
if test_r2_rf > test_r2 + 0.05:
    print("   ✓ Use RANDOM FOREST for production predictions")
    print("   ✓ Use LINEAR MODEL for interpretability and explanations")
elif test_r2_rf > test_r2 + 0.02:
    print("   ≈ Random Forest slightly better but consider tradeoffs")
    print("   • Use RF if accuracy is paramount")
    print("   • Use Linear if interpretability matters more")
else:
    print("   ✓ Use LINEAR MODEL - simpler with similar performance")
    print("   • Random Forest doesn't justify added complexity")

print("\n" + "="*80)
print("✓ RANDOM FOREST MODEL TRAINING AND EVALUATION COMPLETED!")
print("="*80)


RANDOM FOREST REGRESSOR MODEL

⚠️  DATA LEAKAGE ISSUE:
  • Original 'features' column includes SALARY_FROM
  • SALARY_FROM is part of the same salary range as target (SALARY)
  • Using 'features_clean' instead (without SALARY_FROM)

✓ Using 'features_clean' column WITHOUT SALARY_FROM
  Features included: ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE', 'MSA_NAME_VEC', 'REMOTE_TYPE_NAME_VEC']

HYPERPARAMETER SELECTION

📊 HYPERPARAMETER CHOICES:
--------------------------------------------------------------------------------
Number of Trees (numTrees): 200
  • Range: 100-500 trees
  • Rationale: 200 trees provides good ensemble diversity
  • More trees = more stable predictions, longer training time
  • Diminishing returns typically after 200-300 trees

Maximum Depth (maxDepth): 7
  • Range: 4-10 levels
  • Rationale: Depth 7 balances complexity and generalization
  • Deeper trees = more complex patterns, higher overfitting risk
  • Shallower trees = simpler patterns, potential underfit

25/10/08 23:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1456.8 KiB
25/10/08 23:16:47 WARN DAGScheduler: Broadcasting large task binary with size 2040.7 KiB
25/10/08 23:16:49 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


✓ Random Forest model training completed!

RANDOM FOREST MODEL EVALUATION

=== SAMPLE PREDICTIONS (RANDOM FOREST) ===


+------+-----------------+--------------------+--------------------+--------------------+----------------+
|SALARY|       prediction|MIN_YEARS_EXPERIENCE|MAX_YEARS_EXPERIENCE|            MSA_NAME|REMOTE_TYPE_NAME|
+------+-----------------+--------------------+--------------------+--------------------+----------------+
| 49547|77453.18977253485|                   0|                   0|Riverside-San Ber...|          Onsite|
| 41600|97816.58825942391|                   0|                   0|    Jacksonville, FL|          Onsite|
| 66500|78277.45251293929|                   0|                   0|Houston-The Woodl...|          Onsite|
| 48880| 77626.2387262881|                   0|                   0|Denver-Aurora-Lak...|          Onsite|
| 50960|78204.00909099524|                   0|                   0|Chicago-Napervill...|          Onsite|
| 61328| 78276.6119052895|                   0|                   0|Dallas-Fort Worth...|          Onsite|
| 48922|83616.58316176658|           

📊 TRAINING SET PERFORMANCE:
--------------------------------------------------------------------------------
R² (R-squared): 0.4498
  • Interpretation: Model explains 44.98% of variance on training data
RMSE: $27,979.78
  • Average prediction error on training data
MAE: $21,139.79
  • Average absolute error on training data

🎯 TEST SET PERFORMANCE:
--------------------------------------------------------------------------------
R² (R-squared): 0.3851
  • Interpretation: Model explains 38.51% of variance on test data
RMSE: $26,450.42
  • Average prediction error on test data
MAE: $21,263.83
  • Average absolute error on test data

📈 OVERFITTING ANALYSIS:
--------------------------------------------------------------------------------
Training R²: 0.4498
Test R²: 0.3851
Difference: 0.0647
✓ Good generalization - acceptable overfitting

FEATURE IMPORTANCE ANALYSIS



=== TOP 20 MOST IMPORTANT FEATURES ===
             Feature  Importance  Importance_Pct
MIN_YEARS_EXPERIENCE    0.468651       46.865076
MAX_YEARS_EXPERIENCE    0.350747       35.074705
               MSA_0    0.026873        2.687331
              MSA_14    0.024411        2.441104
             MSA_118    0.013282        1.328240
               MSA_4    0.010402        1.040214
              MSA_10    0.010201        1.020104
               MSA_5    0.009260        0.926041
              MSA_12    0.007114        0.711409
            REMOTE_1    0.005811        0.581069
              MSA_17    0.005579        0.557850
               MSA_2    0.004436        0.443565
              MSA_69    0.003242        0.324175
               MSA_1    0.003085        0.308522
              MSA_28    0.002564        0.256447
             MSA_144    0.002520        0.251960
              MSA_13    0.002437        0.243706
               MSA_3    0.002384        0.238402
              MSA_74    0.0022

# Feature Importance Plot

In [ ]:
# ============================================================================
# EXTRACT AND VISUALIZE RANDOM FOREST FEATURE IMPORTANCE
# ============================================================================
print("\n" + "="*80)
print("RANDOM FOREST FEATURE IMPORTANCE VISUALIZATION")
print("="*80 + "\n")

# Extract feature importances from the trained model
feature_importances = rf_model.featureImportances.toArray()

# Get feature names
# Extract actual category names from the pipeline
msa_model = pipeline_model.stages[0]  # msa_indexer
remote_model = pipeline_model.stages[1]  # remote_indexer

msa_labels = msa_model.labels
remote_labels = remote_model.labels

# Create detailed feature names
feature_names_detailed = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE']

# Add MSA features with actual location names (excluding the last one which is baseline)
for i in range(len(msa_labels) - 1):
    feature_names_detailed.append(f'{msa_labels[i]}')

# Add Remote Type features with actual names (excluding the last one which is baseline)
for i in range(len(remote_labels) - 1):
    feature_names_detailed.append(f'{remote_labels[i]}')

print(f"Total features: {len(feature_names_detailed)}")
print(f"Feature importance array length: {len(feature_importances)}")

# Create DataFrame for feature importance
importance_data = []
for i, (name, importance) in enumerate(zip(feature_names_detailed, feature_importances)):
    importance_data.append({
        'Feature': name,
        'Importance': float(importance),
        'Importance_Pct': float(importance) * 100
    })

importance_df = pd.DataFrame(importance_data)
importance_df = importance_df.sort_values('Importance', ascending=False)

# Get top 10 features
top_10 = importance_df.head(10).copy()

print("\n=== TOP 10 MOST IMPORTANT FEATURES ===")
print(top_10.to_string(index=False))

# ============================================================================
# CREATE PLOTLY BAR CHART
# ============================================================================
print("\n" + "="*80)
print("CREATING VISUALIZATION")
print("="*80 + "\n")

# Reverse order for horizontal bar chart (highest at top)
top_10_reversed = top_10.iloc[::-1]

# Create horizontal bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_10_reversed['Feature'],
    x=top_10_reversed['Importance_Pct'],
    orientation='h',
    marker=dict(
        color=top_10_reversed['Importance_Pct'],
        colorscale='Viridis',
        showscale=True,
        colorbar=dict(
            title="Importance %",
            thickness=15,
            len=0.7
        )
    ),
    text=[f'{val:.2f}%' for val in top_10_reversed['Importance_Pct']],
    textposition='outside',
    textfont=dict(size=11),
    hovertemplate='<b>%{y}</b><br>' +
                  'Importance: %{x:.2f}%<br>' +
                  '<extra></extra>'
))

# Update layout
fig.update_layout(
    title={
        'text': 'Top 10 Most Important Features - Random Forest Model',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'family': 'Arial, sans-serif', 'color': '#2c3e50'}
    },
    xaxis_title='Feature Importance (%)',
    yaxis_title='Feature',
    font=dict(size=12, family='Arial, sans-serif'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    height=600,
    width=1000,
    margin=dict(l=250, r=100, t=80, b=80),
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        zeroline=True,
        zerolinecolor='gray',
        zerolinewidth=1
    ),
    yaxis=dict(
        showgrid=False,
        tickfont=dict(size=11)
    ),
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial, sans-serif"
    )
)

# ============================================================================
# SAVE THE PLOT
# ============================================================================
print("Saving plot to file...")

# Create output directory if it doesn't exist
output_dir = '_output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created directory: {output_dir}/")

# Save as PNG
output_path = os.path.join(output_dir, 'rf_feature_importance.png')

# Check if kaleido is available
try:
    import kaleido
    print("✓ Kaleido found")
except ImportError:
    print("❌ Kaleido not found. Please install: pip install kaleido")
    raise

# Save the figure
try:
    fig.write_image(output_path, width=1000, height=600, scale=2)
    print(f"✓ Plot saved successfully to: {output_path}")
except Exception as e:
    print(f"❌ Error: Could not save as PNG: {e}")
    print("\nTroubleshooting steps:")
    print("   1. Restart your Python kernel/session")
    print("   2. Reinstall kaleido: pip uninstall kaleido && pip install kaleido")
    print("   3. Try: pip install -U kaleido plotly")
    raise

# Display the plot (if in interactive environment)
print("\n" + "="*80)
print("FEATURE IMPORTANCE INSIGHTS")
print("="*80 + "\n")

# Calculate cumulative importance
cumulative_importance = 0
print("📊 TOP FEATURES ANALYSIS:\n")
for idx, row in top_10.iterrows():
    cumulative_importance += row['Importance_Pct']
    print(f"{top_10.index.get_loc(idx) + 1}. {row['Feature']}")
    print(f"   Importance: {row['Importance_Pct']:.2f}%")
    print(f"   Cumulative: {cumulative_importance:.2f}%")
    
    # Add interpretation
    if 'EXPERIENCE' in row['Feature']:
        print(f"   → Experience variables are critical predictors")
    elif any(city in row['Feature'] for city in ['New York', 'San Francisco', 'San Jose', 'Seattle', 'Washington']):
        print(f"   → Major tech hub with significant salary premium")
    elif 'Onsite' in row['Feature'] or 'Remote' in row['Feature']:
        print(f"   → Work arrangement impacts salary significantly")
    else:
        print(f"   → Location-specific salary variation")
    print()

print(f"Top 10 features account for {cumulative_importance:.2f}% of total importance")

if cumulative_importance > 50:
    print("\n✓ Top 10 features capture majority of predictive power")
    print("  → Model relies on relatively few key factors")
else:
    print("\n⚠ Top 10 features capture less than 50% of importance")
    print("  → Predictions distributed across many features")

print("\n" + "="*80)
print("✓ FEATURE IMPORTANCE VISUALIZATION COMPLETED!")
print("="*80)

# Note: Interactive display removed for compatibility
# The plot has been saved to the _output/ directory
print(f"\n📁 View the plot at: {output_path}")


RANDOM FOREST FEATURE IMPORTANCE VISUALIZATION

Total features: 214
Feature importance array length: 216

=== TOP 10 MOST IMPORTANT FEATURES ===
                              Feature  Importance  Importance_Pct
                 MIN_YEARS_EXPERIENCE    0.468651       46.865076
                 MAX_YEARS_EXPERIENCE    0.350747       35.074705
New York-Newark-Jersey City, NY-NJ-PA    0.026873        2.687331
   San Jose-Sunnyvale-Santa Clara, CA    0.024411        2.441104
          Omaha-Council Bluffs, NE-IA    0.013282        1.328240
   San Francisco-Oakland-Berkeley, CA    0.010402        1.040214
          Seattle-Tacoma-Bellevue, WA    0.010201        1.020104
   Chicago-Naperville-Elgin, IL-IN-WI    0.009260        0.926041
     Austin-Round Rock-Georgetown, TX    0.007114        0.711409
                               Remote    0.005811        0.581069

CREATING VISUALIZATION

Saving plot to file...
✓ Kaleido found
✓ Plot saved successfully to: _output/rf_feature_importance.png


# Compare 3 Models – GLR, Polynomial, RF

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

# ============================================================================
# GENERATE PREDICTIONS FOR ALL THREE MODELS
# ============================================================================
print("\n" + "="*80)
print("COMPREHENSIVE MODEL COMPARISON")
print("="*80 + "\n")

print("Generating predictions from all three models...")

# Linear Model Predictions (already have test_predictions from earlier)
predictions_linear = lr_model.transform(df_test)

# Polynomial Model Predictions (already have predictions_poly from earlier)
predictions_poly = lr_poly_model.transform(df_test_poly)

# Random Forest Predictions (already have test_predictions_rf from earlier)
predictions_rf = rf_model.transform(df_test)

print("✓ Predictions generated for all three models\n")

# ============================================================================
# CALCULATE PERFORMANCE METRICS FOR ALL MODELS
# ============================================================================
print("="*80)
print("CALCULATING PERFORMANCE METRICS")
print("="*80 + "\n")

# Initialize evaluator
evaluator_rmse = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="rmse")
evaluator_r2 = RegressionEvaluator(labelCol="SALARY", predictionCol="prediction", metricName="r2")

# Calculate RMSE for each model
rmse_linear_test = evaluator_rmse.evaluate(predictions_linear)
rmse_poly_test = evaluator_rmse.evaluate(predictions_poly)
rmse_rf_test = evaluator_rmse.evaluate(predictions_rf)

# Calculate R² for each model
r2_linear_test = evaluator_r2.evaluate(predictions_linear)
r2_poly_test = evaluator_r2.evaluate(predictions_poly)
r2_rf_test = evaluator_r2.evaluate(predictions_rf)

print("📊 RMSE (Root Mean Squared Error):")
print(f"  Linear Model: ${rmse_linear_test:,.2f}")
print(f"  Polynomial Model: ${rmse_poly_test:,.2f}")
print(f"  Random Forest: ${rmse_rf_test:,.2f}")

print("\n📈 R² (R-squared):")
print(f"  Linear Model: {r2_linear_test:.4f}")
print(f"  Polynomial Model: {r2_poly_test:.4f}")
print(f"  Random Forest: {r2_rf_test:.4f}")

# ============================================================================
# CALCULATE AIC AND BIC
# ============================================================================
print("\n" + "="*80)
print("CALCULATING AIC AND BIC")
print("="*80 + "\n")

# For Linear Regression models, we can extract AIC from summary if available
# Otherwise, we calculate manually

def calculate_aic_bic(n, k, rmse):
    """
    Calculate AIC and BIC for regression models
    
    Parameters:
    n: number of observations
    k: number of parameters (features + intercept)
    rmse: root mean squared error
    
    AIC = n*ln(MSE) + 2*k
    BIC = n*ln(MSE) + k*ln(n)
    where MSE = RMSE²
    """
    mse = rmse ** 2
    aic = n * np.log(mse) + 2 * k
    bic = n * np.log(mse) + k * np.log(n)
    return aic, bic

# Get test set size
n_test = df_test.count()

# Get number of parameters for each model
# k = number of features + 1 (intercept)
k_linear = len(coefficients) + 1
k_poly = len(coefficients_poly) + 1
k_rf = len(feature_importances) + 1  # Note: RF doesn't have traditional parameters

print(f"Test set size (n): {n_test}")
print(f"\nNumber of parameters:")
print(f"  Linear Model: {k_linear} (features + intercept)")
print(f"  Polynomial Model: {k_poly} (features + intercept)")
print(f"  Random Forest: {k_rf} (pseudo-parameters for comparison)")

# Calculate AIC and BIC for each model
aic_linear, bic_linear = calculate_aic_bic(n_test, k_linear, rmse_linear_test)
aic_poly, bic_poly = calculate_aic_bic(n_test, k_poly, rmse_poly_test)
aic_rf, bic_rf = calculate_aic_bic(n_test, k_rf, rmse_rf_test)

print("\n📊 AIC (Akaike Information Criterion) - Lower is better:")
print(f"  Linear Model: {aic_linear:,.2f}")
print(f"  Polynomial Model: {aic_poly:,.2f}")
print(f"  Random Forest: {aic_rf:,.2f} (pseudo-AIC)")

print("\n📊 BIC (Bayesian Information Criterion) - Lower is better:")
print(f"  Linear Model: {bic_linear:,.2f}")
print(f"  Polynomial Model: {bic_poly:,.2f}")
print(f"  Random Forest: {bic_rf:,.2f} (pseudo-BIC)")

# Determine best model by each metric
print("\n" + "-" * 80)
print("🏆 BEST MODEL BY EACH METRIC:")
print("-" * 80)

models_dict = {
    'Linear': {'RMSE': rmse_linear_test, 'R²': r2_linear_test, 'AIC': aic_linear, 'BIC': bic_linear},
    'Polynomial': {'RMSE': rmse_poly_test, 'R²': r2_poly_test, 'AIC': aic_poly, 'BIC': bic_poly},
    'Random Forest': {'RMSE': rmse_rf_test, 'R²': r2_rf_test, 'AIC': aic_rf, 'BIC': bic_rf}
}

print(f"  Lowest RMSE: {min(models_dict, key=lambda x: models_dict[x]['RMSE'])}")
print(f"  Highest R²: {max(models_dict, key=lambda x: models_dict[x]['R²'])}")
print(f"  Lowest AIC: {min(models_dict, key=lambda x: models_dict[x]['AIC'])}")
print(f"  Lowest BIC: {min(models_dict, key=lambda x: models_dict[x]['BIC'])}")

# ============================================================================
# CREATE DATAFRAME WITH PREDICTIONS AND ACTUAL VALUES
# ============================================================================
print("\n" + "="*80)
print("CREATING COMPARISON DATAFRAME")
print("="*80 + "\n")

# Collect predictions and actual values from test set
actual_values = np.array(predictions_linear.select('SALARY').collect()).flatten()
pred_linear = np.array(predictions_linear.select('prediction').collect()).flatten()
pred_poly = np.array(predictions_poly.select('prediction').collect()).flatten()
pred_rf = np.array(predictions_rf.select('prediction').collect()).flatten()

# Create comprehensive DataFrame
comparison_df = pd.DataFrame({
    'Actual_Salary': actual_values,
    'Predicted_Linear': pred_linear,
    'Predicted_Polynomial': pred_poly,
    'Predicted_RF': pred_rf,
    'Error_Linear': actual_values - pred_linear,
    'Error_Polynomial': actual_values - pred_poly,
    'Error_RF': actual_values - pred_rf,
    'Abs_Error_Linear': np.abs(actual_values - pred_linear),
    'Abs_Error_Polynomial': np.abs(actual_values - pred_poly),
    'Abs_Error_RF': np.abs(actual_values - pred_rf)
})

print(f"✓ Created comparison DataFrame with {len(comparison_df)} test observations")
print(f"\nDataFrame columns: {list(comparison_df.columns)}")
print(f"\nFirst few rows:")
print(comparison_df.head(10))

# Summary statistics
print("\n" + "-" * 80)
print("PREDICTION SUMMARY STATISTICS:")
print("-" * 80)
print(comparison_df[['Actual_Salary', 'Predicted_Linear', 'Predicted_Polynomial', 'Predicted_RF']].describe())

# ============================================================================
# CREATE 2x2 GRID PLOT: ACTUAL VS PREDICTED FOR EACH MODEL
# ============================================================================
print("\n" + "="*80)
print("CREATING VISUALIZATION: ACTUAL VS PREDICTED")
print("="*80 + "\n")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Calculate min and max for consistent axis ranges
min_val = min(comparison_df['Actual_Salary'].min(), 
              comparison_df['Predicted_Linear'].min(),
              comparison_df['Predicted_Polynomial'].min(),
              comparison_df['Predicted_RF'].min())
max_val = max(comparison_df['Actual_Salary'].max(),
              comparison_df['Predicted_Linear'].max(),
              comparison_df['Predicted_Polynomial'].max(),
              comparison_df['Predicted_RF'].max())

# Add some padding
padding = (max_val - min_val) * 0.05
min_val -= padding
max_val += padding

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        f'Linear Model<br>RMSE: ${rmse_linear_test:,.0f} | R²: {r2_linear_test:.4f}',
        f'Polynomial Model<br>RMSE: ${rmse_poly_test:,.0f} | R²: {r2_poly_test:.4f}',
        f'Random Forest Model<br>RMSE: ${rmse_rf_test:,.0f} | R²: {r2_rf_test:.4f}',
        'Model Comparison Summary'
    ),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "scatter"}, {"type": "table"}]],
    vertical_spacing=0.12,
    horizontal_spacing=0.10
)

# Plot 1: Linear Model (row 1, col 1)
fig.add_trace(
    go.Scatter(
        x=comparison_df['Actual_Salary'],
        y=comparison_df['Predicted_Linear'],
        mode='markers',
        marker=dict(color='#3498db', size=6, opacity=0.6),
        name='Linear',
        hovertemplate='Actual: $%{x:,.0f}<br>Predicted: $%{y:,.0f}<extra></extra>'
    ),
    row=1, col=1
)

# Add perfect prediction line
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(color='red', dash='dash', width=2),
        name='Perfect Prediction',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

# Plot 2: Polynomial Model (row 1, col 2)
fig.add_trace(
    go.Scatter(
        x=comparison_df['Actual_Salary'],
        y=comparison_df['Predicted_Polynomial'],
        mode='markers',
        marker=dict(color='#e74c3c', size=6, opacity=0.6),
        name='Polynomial',
        hovertemplate='Actual: $%{x:,.0f}<br>Predicted: $%{y:,.0f}<extra></extra>'
    ),
    row=1, col=2
)

# Add perfect prediction line
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(color='red', dash='dash', width=2),
        name='Perfect Prediction',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=2
)

# Plot 3: Random Forest Model (row 2, col 1)
fig.add_trace(
    go.Scatter(
        x=comparison_df['Actual_Salary'],
        y=comparison_df['Predicted_RF'],
        mode='markers',
        marker=dict(color='#2ecc71', size=6, opacity=0.6),
        name='Random Forest',
        hovertemplate='Actual: $%{x:,.0f}<br>Predicted: $%{y:,.0f}<extra></extra>'
    ),
    row=2, col=1
)

# Add perfect prediction line
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        line=dict(color='red', dash='dash', width=2),
        name='Perfect Prediction',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=2, col=1
)

# Plot 4: Summary Table (row 2, col 2)
# Determine best model for highlighting
best_rmse_idx = [rmse_linear_test, rmse_poly_test, rmse_rf_test].index(min(rmse_linear_test, rmse_poly_test, rmse_rf_test))
best_r2_idx = [r2_linear_test, r2_poly_test, r2_rf_test].index(max(r2_linear_test, r2_poly_test, r2_rf_test))
best_aic_idx = [aic_linear, aic_poly, aic_rf].index(min(aic_linear, aic_poly, aic_rf))
best_bic_idx = [bic_linear, bic_poly, bic_rf].index(min(bic_linear, bic_poly, bic_rf))

# Create color coding for best values
rmse_colors = ['lightgreen' if i == best_rmse_idx else 'white' for i in range(3)]
r2_colors = ['lightgreen' if i == best_r2_idx else 'white' for i in range(3)]
aic_colors = ['lightgreen' if i == best_aic_idx else 'white' for i in range(3)]
bic_colors = ['lightgreen' if i == best_bic_idx else 'white' for i in range(3)]

fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Metric</b>', '<b>Linear</b>', '<b>Polynomial</b>', '<b>Random Forest</b>'],
            fill_color='#34495e',
            font=dict(color='white', size=12),
            align='left',
            height=30
        ),
        cells=dict(
            values=[
                ['RMSE', 'R²', 'AIC', 'BIC', 'Parameters'],
                [f'${rmse_linear_test:,.0f}', f'{r2_linear_test:.4f}', f'{aic_linear:,.0f}', 
                 f'{bic_linear:,.0f}', f'{k_linear}'],
                [f'${rmse_poly_test:,.0f}', f'{r2_poly_test:.4f}', f'{aic_poly:,.0f}', 
                 f'{bic_poly:,.0f}', f'{k_poly}'],
                [f'${rmse_rf_test:,.0f}', f'{r2_rf_test:.4f}', f'{aic_rf:,.0f}', 
                 f'{bic_rf:,.0f}', f'{k_rf}']
            ],
            fill_color=[
                ['white', 'white', 'white', 'white', 'white'],
                rmse_colors[0:1] + r2_colors[0:1] + aic_colors[0:1] + bic_colors[0:1] + ['white'],
                rmse_colors[1:2] + r2_colors[1:2] + aic_colors[1:2] + bic_colors[1:2] + ['white'],
                rmse_colors[2:3] + r2_colors[2:3] + aic_colors[2:3] + bic_colors[2:3] + ['white']
            ],
            align='left',
            font=dict(size=11),
            height=25
        )
    ),
    row=2, col=2
)

# Update axes for scatter plots
for row in [1, 2]:
    for col in [1, 2]:
        if not (row == 2 and col == 2):  # Skip the table subplot
            fig.update_xaxes(
                title_text="Actual Salary ($)",
                showgrid=True,
                gridcolor='lightgray',
                range=[min_val, max_val],
                row=row, col=col
            )
            fig.update_yaxes(
                title_text="Predicted Salary ($)",
                showgrid=True,
                gridcolor='lightgray',
                range=[min_val, max_val],
                row=row, col=col
            )

# Update overall layout
fig.update_layout(
    title={
        'text': 'Model Comparison: Actual vs Predicted Salary',
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#2c3e50'}
    },
    showlegend=False,
    height=900,
    width=1400,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(family='Arial, sans-serif', size=11)
)

# Save the plot
import os
output_dir = '_output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_path = os.path.join(output_dir, 'model_comparison_actual_vs_predicted.png')

try:
    import kaleido
    print("✓ Kaleido found")
except ImportError:
    print("❌ Kaleido not found. Please install: pip install kaleido")

try:
    fig.write_image(output_path, width=1400, height=900, scale=2)
    print(f"✓ Plot saved to: {output_path}")
except Exception as e:
    print(f"⚠️  Could not save PNG: {e}")
    print("   Displaying plot instead...")
    fig.show()

# ============================================================================
# CREATE METRICS COMPARISON TABLE
# ============================================================================
print("\n" + "="*80)
print("FINAL METRICS COMPARISON TABLE")
print("="*80 + "\n")

metrics_comparison = pd.DataFrame({
    'Model': ['Linear', 'Polynomial', 'Random Forest'],
    'RMSE': [rmse_linear_test, rmse_poly_test, rmse_rf_test],
    'R²': [r2_linear_test, r2_poly_test, r2_rf_test],
    'AIC': [aic_linear, aic_poly, aic_rf],
    'BIC': [bic_linear, bic_poly, bic_rf],
    'Parameters': [k_linear, k_poly, k_rf]
})

print(metrics_comparison.to_string(index=False))

# Highlight best values
print("\n" + "-" * 80)
print("BEST PERFORMING MODEL BY METRIC:")
print("-" * 80)
print(f"  Best RMSE (lowest): {metrics_comparison.loc[metrics_comparison['RMSE'].idxmin(), 'Model']}")
print(f"  Best R² (highest): {metrics_comparison.loc[metrics_comparison['R²'].idxmax(), 'Model']}")
print(f"  Best AIC (lowest): {metrics_comparison.loc[metrics_comparison['AIC'].idxmin(), 'Model']}")
print(f"  Best BIC (lowest): {metrics_comparison.loc[metrics_comparison['BIC'].idxmin(), 'Model']}")

print("\n" + "="*80)
print("✓ COMPREHENSIVE MODEL COMPARISON COMPLETED!")
print("="*80)


COMPREHENSIVE MODEL COMPARISON

Generating predictions from all three models...
✓ Predictions generated for all three models

CALCULATING PERFORMANCE METRICS



📊 RMSE (Root Mean Squared Error):
  Linear Model: $26,948.87
  Polynomial Model: $26,723.22
  Random Forest: $26,450.42

📈 R² (R-squared):
  Linear Model: 0.3617
  Polynomial Model: 0.3723
  Random Forest: 0.3851

CALCULATING AIC AND BIC



Test set size (n): 1022

Number of parameters:
  Linear Model: 217 (features + intercept)
  Polynomial Model: 218 (features + intercept)
  Random Forest: 217 (pseudo-parameters for comparison)

📊 AIC (Akaike Information Criterion) - Lower is better:
  Linear Model: 21,286.27
  Polynomial Model: 21,271.08
  Random Forest: 21,248.11 (pseudo-AIC)

📊 BIC (Bayesian Information Criterion) - Lower is better:
  Linear Model: 22,355.97
  Polynomial Model: 22,345.72
  Random Forest: 22,317.81 (pseudo-BIC)

--------------------------------------------------------------------------------
🏆 BEST MODEL BY EACH METRIC:
--------------------------------------------------------------------------------
  Lowest RMSE: Random Forest
  Highest R²: Random Forest
  Lowest AIC: Random Forest
  Lowest BIC: Random Forest

CREATING COMPARISON DATAFRAME



✓ Created comparison DataFrame with 1022 test observations

DataFrame columns: ['Actual_Salary', 'Predicted_Linear', 'Predicted_Polynomial', 'Predicted_RF', 'Error_Linear', 'Error_Polynomial', 'Error_RF', 'Abs_Error_Linear', 'Abs_Error_Polynomial', 'Abs_Error_RF']

First few rows:
   Actual_Salary  Predicted_Linear  Predicted_Polynomial  Predicted_RF  \
0          49547      54123.724108          48293.287526  77453.189773   
1          41600      81030.786228          70945.633303  97816.588259   
2          66500      77147.764021          69672.375195  78277.452513   
3          48880      70845.729707          63514.959106  77626.238726   
4          50960      79838.892875          71940.144437  78204.009091   
5          61328      75431.718778          68048.234626  78276.611905   
6          48922      71006.566573          64574.175973  83616.583162   
7          62400      81505.588101          73590.608769  81171.545411   
8          62400      84839.101324          77046.61

# Appendix: AI Prompts

In [11]:
from IPython.display import HTML

with open("assignment04-avagodsy-AIPrompts.html") as f:
    display(HTML(f.read()))